In [ ]:
from neo4j import GraphDatabase

# Define the connection details
uri =       "neo4j://localhost:7687"
username =  "neo4j"
password =  "password"


# Connect to the Neo4j database
driver = GraphDatabase.driver(uri, auth=(username, password))

# figure

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
!pip install python-louvain
import networkx as nx
import community
from community import community_louvain
import matplotlib.pyplot as plt

layout_algorithms = [
    nx.circular_layout,
    nx.random_layout,
    nx.spring_layout,
    nx.spectral_layout,
    nx.shell_layout,
    nx.bipartite_layout,
    nx.planar_layout,
    nx.fruchterman_reingold_layout,
    nx.kamada_kawai_layout
]

A=['PGM1-CDG','Cerebral cavernous malformations 3',"CLOVES syndrome","STT3B-CDG","Thoracolaryngopelvic dysplasia","Actinic prurigo","Non-syndromic polydactyly","Genitopatellar syndrome","Diaphanospondylodysostosis"]
B=['GARD0004329','GARD0018314',"GARD0010939","GARD0017603","GARD0005184", "GARD0017510","GARD0004410","GARD0010994","GARD0016674"]
for n in range(len(A)):
  disease= A[n]  #'Actinic prurigo' #'Non-syndromic polydactyly'   #'Cystic fibrosis'
  diseaseid=B[n]

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def get_reseracher_info(disease):
 with driver.session() as session:
  query = "MATCH (i:Researcher)--(C:Cluster)--(g:Gard) WHERE g.GARDname=$disease OPTIONAL MATCH (i)-[:Collaborated_With]-(n:Researcher)  WITH i, COLLECT(n.Name + ', @institution: ' + n.Aff_name) AS collaborators RETURN i.Name + ', @institution: ' + i.Aff_name AS researcher, collaborators "
  results = session.run(query, disease=disease)
  data = [record.data() for record in results]  # Convert result to a list of dictionaries
  reseracher_info = pd.DataFrame(data)
 return reseracher_info


In [ ]:
import pandas as pd
disease='Cystic fibrosis'
#cluster_info=get_cluster(disease)
reseracher_info=get_reseracher_info(disease)
#expertise=get_expertise_info(disease)
#cluster_info
#reseracher_info

In [ ]:
import numpy as np
def draw_graph(disease, HA="center", VA="center"):
 #cluster_info=get_cluster(disease)
 #reseracher_info=get_reseracher_info(disease)
 #expertise=get_expertise_info(disease)

 # Create a directed graph
 G = nx.Graph()
 reseracher_info1=reseracher_info[:2000]
 for i in reseracher_info1.index:
    new_node = reseracher_info1['researcher'][i]
    if new_node not in G:
        G.add_node(new_node)
    collab = reseracher_info1['collaborators'][i]
    for edge in collab:
        if edge not in G:
            G.add_node(edge)
        if not G.has_edge(new_node, edge):
            G.add_edge(new_node, edge)

 #partition = cluster_info
 #nodes_to_remove = [node for node, attr in G.nodes(data=True) if node not in cluster_info]
 #for node in nodes_to_remove:
 #   if node in G:
 #       G.remove_node(node)

 # Draw the graph with nodes colored by their community
 #pos = nx.kamada_kawai_layout(G)
 #cmap = plt.cm.get_cmap('viridis', max(partition.values()) + 1)

 partition = community_louvain.best_partition(G)
 #print(partition)
 pos = nx.kamada_kawai_layout(G)
 cmap = plt.cm.get_cmap('viridis', max(partition.values()) + 1)
 node_colors = [cmap(partition[node]) for node in G.nodes()]
 nx.draw_networkx_nodes(G, pos, node_size=20, node_color=node_colors,  alpha=0.8)


 #pos = nx.spring_layout(G, k=0.1, iterations=50, seed=42)
 #cmap = plt.cm.get_cmap('viridis', max(partition.values()) + 1)
 #node_colors = [cmap(partition[node]) for node in G.nodes()]
 #nx.draw_networkx_nodes(G, pos, node_size=20, node_color=node_colors, alpha=0.8)


 #nx.draw_networkx_nodes(G, pos, node_size=20, node_color=[cmap(partition[node]) for node in G.nodes()], alpha=0.8)
 nx.draw_networkx_edges(G, pos, edge_color="gray", width=0.2, alpha=0.6)
 '''
 # Label one node per community
 community_labels = {}
 for node, community_id in partition.items():
    if community_id not in community_labels:
        community_labels[community_id] = node

 # Set figure limits
 x_vals, y_vals = zip(*pos.values())
 x_min, x_max = min(x_vals), max(x_vals)
 y_min, y_max = min(y_vals), max(y_vals)
 x_margin = (x_max - x_min)*0.1 #* 0.3
 y_margin = (y_max - y_min)*0.1 #* 0.3

 #ax.set_frame_on(False)  # Disable figure border
 plt.axis('off')
 # Organize labels to avoid overlaps
 for community_id, node in community_labels.items():
  #if community_id>1:
    x, y = pos[node]

    # Apply an offset for the labels
    offset = 0.1 + 0.2 * community_id
    x_label = x + offset * np.cos(community_id)
    y_label = y + offset * np.sin(community_id)

    # Clamp label positions within the figure bounds
    x_label = max(x_min - x_margin, min(x_max + x_margin, x_label))
    y_label = max(y_min - y_margin, min(y_max + y_margin, y_label))

    # Display label with expertise information
    try:
        label_text = (
            f"C{community_id}: {expertise[community_id][:25]}\n"
            f"{expertise[community_id][25:50]}\n"
            f"{expertise[community_id][50:75]}\n"
            f"{expertise[community_id][75:100]}\n"
            f"{expertise[community_id][100:125]}..."
        )
        plt.text(
            x_label, y_label, label_text, fontsize=6, color="green", ha=HA, va=VA,
            bbox=dict(facecolor="white", alpha=0.6, edgecolor="red", boxstyle="round,pad=0.3")
        )
    except:
        pass
 '''
 # Finalize plot
 #plt.title("Social Network for Rare Disease-Based Research Collaboration Recommendation")
 plt.xlim(x_min - x_margin, x_max + x_margin)
 plt.ylim(y_min - y_margin, y_max + y_margin)
 #plt.xlim(x_min, x_max )
 #plt.ylim(y_min , y_max )
 plt.show()
draw_graph('Cystic fibrosis', HA="right", VA="center")


KeyboardInterrupt



CALL db.schema.visualization()

# First part

In [ ]:
# Define node types
node_types = ['Researcher',
'Cluster',
'Location',
'Expertise',
'Gard',
'ClinicalTrial',
'Grant',
'Pubmed']

# Create nodes
with driver.session() as session:
    for node_type in node_types:
        query = f"CREATE (:`{node_type}`)"
        session.run(query)

In [ ]:
import pandas as pd
#ClinicalTrial = pd.read_csv('C:/Users/Desktop/Social network all works/work rdas/Clinicaltrial.csv')
#Grant = pd.read_csv('C:/Users/Desktop/Social network all works/work rdas/Grant.csv')#,  error_bad_lines=False)
#Pubmed = pd.read_csv('C:/Users/Desktop/Social network all works/work rdas/Pubmed.csv', on_bad_lines='skip')# on_bad_lines='skip') #error_bad_lines=False

ClinicalTrial = pd.read_csv('C:/Users/valinejadj2/Desktop/ClinicalTrial.csv')
Grant = pd.read_csv('C:/Users/Desktop/Grants.csv')#,  error_bad_lines=False)
data1=pd.read_csv('C:/Users/Desktop/pubmed1.csv')
data2=pd.read_csv('C:/Users/Desktop/pubmed2.csv')
data3=pd.read_csv('C:/Users/Desktop/pubmed3.csv')
data4=pd.read_csv('C:/Users/Desktop/Pubmed.csv')
data=pd.concat([data1,data2,data3,data4], ignore_index=True)
Pubmed = data.drop_duplicates()
Pubmed.index= range(Pubmed.shape[0])
ClinicalTrial = ClinicalTrial.replace('"', '', regex=True)
Grant = Grant.replace('"', '', regex=True)
Pubmed = Pubmed.replace('"', '', regex=True)
print(ClinicalTrial.shape[0], Grant.shape[0], Pubmed.shape[0])

1617 14033 215248


In [ ]:
num_nulls = Pubmed['Abstract'].isnull().sum()
print(f"Number of nulls in 'c.BriefSummary': {num_nulls}")
num_empty_strings = (Pubmed['Abstract'] == '').sum()
print(f"Number of empty strings in 'c.BriefSummary': {num_empty_strings}")

Number of nulls in 'c.BriefSummary': 9
Number of empty strings in 'c.BriefSummary': 14945


In [ ]:
print(ClinicalTrial.columns)
print(Grant.columns)
print(Pubmed.columns)

Index(['i.OfficialName', 'i.OfficialAffiliation', 'c. BriefTitle',
       'c.OfficialTitle', 'c.BriefSummary', 'c.NCTId', 'l.LocationCity',
       'l. LocationState', 'l.LocationCountry', 'l.LocationFacility',
       'Interventions'],
      dtype='object')
Index(['pi.pi_name', 'pi.org_name', 'p.title', 'p.application_id',
       'p.abstract', 'collect(distinct p.terms)'],
      dtype='object')
Index(['Title', 'PubMedID', 'Abstract', 'Keywords', 'AuthorFullName',
       'AuthorAffiliation'],
      dtype='object')


In [ ]:
ClinicalTrial['GARDname']= 'Cystic fibrosis'
Grant['GARDname']= 'Cystic fibrosis'
Pubmed['GARDname']= 'Cystic fibrosis'

In [ ]:
rename_dict_1 = {
    'i.OfficialName':         'OfficialName',
    'i.OfficialAffiliation':  'OfficialAffiliation',
    'c. BriefTitle':       'BriefTitle',
    'c.OfficialTitle':    'OfficialTitle',
    'c.BriefSummary':     'BriefSummary',
    'c.NCTId':            'NCTId',
    'l.LocationCity':     'LocationCity',
    'l. LocationState':   'LocationState',
    'l.LocationCountry':  'LocationCountry',
    'l.LocationFacility': 'LocationFacility' }

rename_dict_2 = {
    'pi.pi_name':           'pi_name',
    'pi.org_name':          'org_name',
    'p.title':              'title',
    'p.application_id':     'application_id',
    'p.abstract':           'abstract',
    'collect(distinct p.terms)':              'terms'
}

rename_dict_3 = {
    'AuthorFullName':            'fullName'        ,
    'Title':              'title'       ,
    'AuthorAffiliation':        'affiliation'       ,
    'PubMedID':          'pubmed_id'      ,
    'Abstract':       'abstractText'     ,
    'Keywords':  'concatenatedKeywords'
}


# Rename the columns of each DataFrame
ClinicalTrial.rename(columns=rename_dict_1, inplace=True)
Grant.rename(columns=rename_dict_2, inplace=True)
Pubmed.rename(columns=rename_dict_3, inplace=True)

# Check the renamed columns
print("DF1 columns:", ClinicalTrial.columns)
print("DF2 columns:", Grant.columns)
print("DF3 columns:", Pubmed.columns)

DF1 columns: Index(['OfficialName', 'OfficialAffiliation', 'BriefTitle', 'OfficialTitle',
       'BriefSummary', 'NCTId', 'LocationCity', 'LocationState',
       'LocationCountry', 'LocationFacility', 'Interventions', 'GARDname'],
      dtype='object')
DF2 columns: Index(['pi_name', 'org_name', 'title', 'application_id', 'abstract', 'terms',
       'GARDname'],
      dtype='object')
DF3 columns: Index(['title', 'pubmed_id', 'abstractText', 'concatenatedKeywords',
       'fullName', 'affiliation', 'GARDname'],
      dtype='object')


In [ ]:
def create_Gard_node(properties):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "create (ga:Gard {"
                'GARDID: $GARDID,'
                'GARDname: $GARDname'
                "})"
            )
            session.run(query, **properties)
def create_ClinicalTrial_node(properties):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "create (CT:ClinicalTrial {"
                'OfficialName: $OfficialName,'
                'OfficialAffiliation: $OfficialAffiliation,'
                'BriefTitle: $BriefTitle,'
                'OfficialTitle: $OfficialTitle,'
                'BriefSummary: $BriefSummary,'
                'NCTId: $NCTId,'
                'LocationCity: $LocationCity,'
                'LocationState: $LocationState,'
                'LocationCountry: $LocationCountry,'
                'LocationFacility: $LocationFacility,'
                'Interventions: $Interventions,'
                'GARDname: $GARDname'
                "})"
            )
            session.run(query, **properties)
'''
def create_ClinicalTrial_node(properties):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "MERGE (CT:ClinicalTrial {"
                'NCTId: $NCTId'  # Unique identifier for the ClinicalTrial
                "}) "
                "ON CREATE SET "
                'CT.OfficialName = $OfficialName, '
                'CT.OfficialAffiliation = $OfficialAffiliation, '
                'CT.BriefTitle = $BriefTitle, '
                'CT.OfficialTitle = $OfficialTitle, '
                'CT.BriefSummary = $BriefSummary, '
                'CT.LocationCity = $LocationCity, '
                'CT.LocationState = $LocationState, '
                'CT.LocationCountry = $LocationCountry, '
                'CT.LocationFacility = $LocationFacility, '
                'CT.Interventions = $Interventions, '
                'CT.GARDname = $GARDname'
            )
            session.run(query, **properties)
'''
def create_Grant_node(properties):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "create (gr:Grant {"
    'pi_name: $pi_name,'
    'org_name: $org_name,'
    'title: $title,'
    'application_id: $application_id,'
    'abstract: $abstract,'
    'terms: $terms,'
    'GARDname: $GARDname'
                "})"
            )
            session.run(query, **properties)
def create_Pubmed_node(properties):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "create (pm:Pubmed {"
    'fullName: $fullName,'
    'ar_title: $ar_title,'
    'ar_affiliation: $ar_affiliation,'
    'pubmed_id: $pubmed_id,'
    'abstractText: $abstractText,'
    'concatenatedKeywords: $concatenatedKeywords,'
    'GARDname: $GARDname'
                "})"
            )
            session.run(query, **properties)

In [ ]:
for i in range(len(ClinicalTrial)):
      event_properties = {
                'OfficialName': ClinicalTrial['OfficialName'][i],
                'OfficialAffiliation': ClinicalTrial['OfficialAffiliation'][i],
                'BriefTitle': ClinicalTrial['BriefTitle'][i],
                'OfficialTitle':ClinicalTrial['OfficialTitle'][i],
                'BriefSummary': ClinicalTrial['BriefSummary'][i],
                'NCTId': ClinicalTrial['NCTId'][i],
                'LocationCity': ClinicalTrial['LocationCity'][i],
                 'LocationState': ClinicalTrial['LocationState'][i],
                'LocationCountry': ClinicalTrial['LocationCountry'][i],
                'LocationFacility': ClinicalTrial['LocationFacility'][i],
                'Interventions': ClinicalTrial['Interventions'][i],
                'GARDname': ClinicalTrial['GARDname'][i]
        }
      create_ClinicalTrial_node(event_properties)
print('CT: finished')

for i in range(len(Grant)):
    event_properties = {
        'pi_name': str(Grant['pi_name'][i]),
        'org_name': str(Grant['org_name'][i]),
        'title': str(Grant['title'][i]),
        'application_id': str(Grant['application_id'][i]),
        'abstract': str(Grant['abstract'][i]),
        'terms': str(Grant['terms'][i]),
        'GARDname': Grant['GARDname'][i]
    }
    create_Grant_node(event_properties)
print('Grant: finished')

In [ ]:
for i in range(len(Pubmed)):
    event_properties = {
        'fullName': Pubmed['fullName'][i],  # Convert to string
        'ar_title': Pubmed['title'][i],  # Convert to string
        'ar_affiliation': Pubmed['affiliation'][i],  # Convert to string
        'pubmed_id': Pubmed['pubmed_id'][i],  # Convert to string (or other appropriate type)
        'abstractText': Pubmed['abstractText'][i],  # Convert to string
        'concatenatedKeywords': Pubmed['concatenatedKeywords'][i],  # Convert to string
        'GARDname': Pubmed['GARDname'][i]  # Convert to string
    }
    create_Pubmed_node(event_properties)

print('Pubmed: finished')


Pubmed: finished


In [ ]:
def remove_duplicate_gard_nodes(uri, username, password):
    query = """
    MATCH (ga:Gard)
    WITH ga.GARDID AS id, COLLECT(ga) AS nodes
    WHERE SIZE(nodes) > 1
    FOREACH (n IN TAIL(nodes) | DELETE n)
    """

    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            session.run(query)

# Usage
remove_duplicate_gard_nodes(uri, username, password)

In [ ]:
def delete_all_gard_nodes():
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            session.run("MATCH (g:Gard) DETACH DELETE g")

delete_all_gard_nodes()


In [ ]:
event_properties = {'GARDID': 'GARD0006233' ,'GARDname': 'Cystic fibrosis'}
create_Gard_node(event_properties)

In [ ]:
def create_relationship_between_nodes(node1_label, node2_label, relationship_type):
    with driver.session() as session:
        query = (
        f"MATCH (n1:{node1_label}), (n2:{node2_label}) "
         f"WHERE n1.GARDname = n2.GARDname "
       f"CREATE (n1)-[:{relationship_type}]->(n2)"
        )
        session.run(query)


create_relationship_between_nodes('ClinicalTrial','Gard',  "RELATED_GARD")
create_relationship_between_nodes('Grant','Gard',  "RELATED_GARD")
create_relationship_between_nodes('Pubmed','Gard',  "RELATED_GARD")

In [ ]:
import pandas as pd
fips_finding=pd.read_csv('C:/Users/valinejadj2/Desktop/Social network all works/work rdas/uscities.csv')
fips_finding_dict=dict()
for i in fips_finding.index:
     fips_finding_dict[(fips_finding['city'][i].lower().strip(),fips_finding['state_name'][i].lower().strip() )]= ['{:0>5}'.format(fips_finding['county_fips'][i]),fips_finding['county_name'][i]]
     fips_finding_dict[fips_finding['city'][i].lower().strip()]= ['{:0>5}'.format(fips_finding['county_fips'][i]),fips_finding['county_name'][i]]

def find_fip(x,y):
 if y != None:
  try:
    return (fips_finding_dict[ (x.lower().strip(),y.lower().strip())][0]), fips_finding_dict[ (x.lower(),y.lower())][1]
  except:
    return  '',''
 else:
    try:
      return (fips_finding_dict[x.lower().strip()][0]),fips_finding_dict[ x.lower()][1]
    except:
      return  '',''

In [ ]:
def create_Researcher_node(properties):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "MERGE (Re:Researcher {"
                    'Name: $Name,'
                    'Aff_name: $Aff_name'
                "}) "
                "ON CREATE SET "
                    "Re.Contact_info= $Contact_info,"
                    "Re.Aff_Zip = $Aff_Zip "

            )
            result = session.run(query, **properties)


def create_Location_node(properties, Name, Aff_name):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "MERGE (Loc:Location {Aff_country: $Aff_country, Aff_state: $Aff_state, Aff_county: $Aff_county}) "
                "ON CREATE SET "
                "Loc.Aff_city = $Aff_city, "
                "Loc.Aff_FIPS = $Aff_FIPS "
                "WITH Loc "
                "MATCH (Re:Researcher {Name: $Name, Aff_name: $Aff_name}) "
                "CREATE (Re)-[:Researcher_location]->(Loc)"
            )
            # Execute the query, passing the properties dictionary and other parameters
            result = session.run(query, **properties, Name=Name, Aff_name=Aff_name)


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")


def classify_entity(text):
    if not isinstance(text, str):  # Check if the input is a string
        return False
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return True
    return False

display(classify_entity('Devon Leadman'))
classify_entity('NIH')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------- -------------------------------- 2.4/12.8 MB 13.4 MB/s eta 0:00:01
     ---------------- ----------------------- 5.2/12.8 MB 13.3 MB/s eta 0:00:01
     ---------------------- ----------------- 7.3/12.8 MB 12.2 MB/s eta 0:00:01
     --------------------------- ------------ 8.9/12.8 MB 11.1 MB/s eta 0:00:01
     ------------------------------- ------- 10.2/12.8 MB 10.3 MB/s eta 0:00:01
     ---------------------------------- ----- 11.0/12.8 MB 9.2 MB/s eta 0:00:01
     ------------------------------------ --- 11.8/12.8 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 7.9 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 7.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now

True

False

In [ ]:
def extract_and_create_researcher_location_nodes_CT():
 with driver.session() as session:
  query = "MATCH (i:ClinicalTrial) RETURN i.OfficialName AS OfficialName, i.OfficialAffiliation AS OfficialAffiliation, i.LocationCountry as LocationCountry, i.LocationCity as LocationCity,i.LocationState as LocationState"
  results = session.run(query)
  for ClinicalTrial in results:
   Name=  ClinicalTrial['OfficialName']
   Aff_name=   str(ClinicalTrial['OfficialAffiliation'])
   if  classify_entity(Name)==True and (Name, Aff_name) not in current_names:
    event_properties = {
        'Name': Name,
        'Contact_info': '',#str(ClinicalTrial['i.OfficialAffiliation'][i]),
        'Aff_name':Aff_name,
        'Aff_Zip': ''#str(ClinicalTrial['LocationZip'][i]),
    }
    create_Researcher_node(event_properties)
    city_  = str(ClinicalTrial['LocationCity']).lower().strip()
    State_ = str(ClinicalTrial['LocationState']).lower().strip()
    event_properties = {
        'Aff_country': str(ClinicalTrial['LocationCountry']),
        'Aff_state': State_,
        'Aff_county': find_fip(city_, State_ )[1] ,
        'Aff_city': city_,
        'Aff_FIPS': find_fip(city_, State_ )[0]
    }
    create_Location_node(event_properties, Name, Aff_name)
    current_names[(Name, Aff_name)] =1

current_names={}
extract_and_create_researcher_location_nodes_CT()

In [ ]:
def extract_and_create_researcher_location_nodes_CT():
 with driver.session() as session:
  query = "MATCH (i:ClinicalTrial) RETURN i.OfficialName AS OfficialName, i.OfficialAffiliation AS OfficialAffiliation, i.LocationCountry as LocationCountry, i.LocationCity as LocationCity,i.LocationState as LocationState"
  results = session.run(query)
  for ClinicalTrial in results:
   Name=  ClinicalTrial['OfficialName']
   Aff_name=   str(ClinicalTrial['OfficialAffiliation'])
   if  classify_entity(Name)==True and (Name, Aff_name) not in current_names:
    event_properties = {
        'Name': Name,
        'Contact_info': '',#str(ClinicalTrial['i.OfficialAffiliation'][i]),
        'Aff_name':Aff_name,
        'Aff_Zip': ''#str(ClinicalTrial['LocationZip'][i]),
    }
    create_Researcher_node(event_properties)
    city_  = str(ClinicalTrial['LocationCity']).lower().strip()
    State_ = str(ClinicalTrial['LocationState']).lower().strip()
    event_properties = {
        'Aff_country': str(ClinicalTrial['LocationCountry']),
        'Aff_state': State_,
        'Aff_county': find_fip(city_, State_ )[1] ,
        'Aff_city': city_,
        'Aff_FIPS': find_fip(city_, State_ )[0]
    }
    create_Location_node(event_properties, Name, Aff_name)
    current_names[(Name, Aff_name)] =1

current_names={}
extract_and_create_researcher_location_nodes_CT()

In [ ]:
#!pip install torch torchvision torchaudio
!pip install locationtagger
!pip install spacy
import nltk
import spacy
# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')
!python -m spacy download en_core_web_sm
!pip install lxml[html_clean]
import locationtagger

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\valinejadj2\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\valinejadj2\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\valinejadj2\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\valinejadj2\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\valinejadj2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\valinejadj2\AppData\Roaming\nltk

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 7.1 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 7.4 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 5.9 MB/s eta 0:00:02
     ------------- -------------------------- 4.2/12.8 MB 5.7 MB/s eta 0:00:02
     ----------------- ---------------------- 5.5/12.8 MB 5.8 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 5.4 MB/s eta 0:00:02
     ---------------------- ----------------- 7.1/12.8 MB 5.1 MB/s eta 0:00:02
     ------------------------ --------------- 7.9/12.8 MB 5.0 MB/s eta 0:00:01
     --------------------------- ------------ 8.9/12.8 MB 5.0 MB/s eta 0:00:01
     ------------------------------- -------- 10.0/12.8 MB 5.1 MB/s eta 0:00:01
     --------------------------------- ------ 10.7/12.8 MB 

In [ ]:
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida",
    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine",
    "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska",
    "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio",
    "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas",
    "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]
states_lower = [state.lower() for state in states]
states_lower_set = set(state.lower() for state in states)

def location_information(j):
 if j:
    place_entity = locationtagger.find_locations(text=j)
    if place_entity.countries and place_entity.countries[0] != 'United States':
       if place_entity.countries:country = place_entity.countries[0]
       else:country = None  # or some default value
       if place_entity.regions:region = place_entity.regions[0]
       else: region = None  # or some default value
       if place_entity.cities: city = place_entity.cities[0]
       else:city = None  # or some default value
       return country, region, city
    else:
     if place_entity.countries or (place_entity.regions and place_entity.regions[0].lower() in states_lower_set):
        # If country is specified as US or region matches a state, proceed
        city_lower=None
        for i in reversed(place_entity.cities):
            city_lower = i.lower()
            if city_lower in fips_finding_dict and city_lower not in ['hospital', 'university']:
                   break
        if place_entity.countries:country = place_entity.countries[0]
        else:country = None  # or some default value
        if place_entity.regions:region = place_entity.regions[0]
        else: region = None  # or some default value
        if city_lower: city = city_lower
        else:city = None  # or some default value
        return country, region, city
    return None, None, None
 return None, None, None

def extract_and_create_researcher_location_nodes_PubMed():
 with GraphDatabase.driver(uri, auth=(username, password)) as driver:

  with driver.session() as session:
   query = "MATCH (i:Pubmed) RETURN i.fullName AS fullName, i.ar_affiliation AS affiliation"
   results = session.run(query)
   for Pubmed in results:
    A,B=Pubmed['fullName'], str(Pubmed['affiliation']).replace('[','').replace(']','')
    if  (A, B) not in current_names:
     if B in pubmed_institution:
        place_entity=pubmed_institution[B]
     else:
       place_entity =location_information(B)
       pubmed_institution[B]= place_entity

     event_properties = {
        'Name': str(A),
        'Contact_info': '',#str(Pubmed['i.OfficialAffiliation'][i]),
        'Aff_name': B,
        'Aff_Zip': ''  }
     create_Researcher_node(event_properties)
     current_names[(A, B)] =1
     if place_entity[0] != None and place_entity[1] !=  None and place_entity[2] !=  None:
      event_properties = {
         'Aff_country': place_entity[0],
        'Aff_state':   place_entity[1],
        'Aff_county': find_fip(place_entity[2], place_entity[1] )[1],
        'Aff_city': place_entity[2],
        'Aff_FIPS': find_fip(place_entity[2], place_entity[1] )[0]  }
      create_Location_node(event_properties, A, B)

pubmed_institution={}
current_names={}
extract_and_create_researcher_location_nodes_PubMed()

In [ ]:
import pandas as pd
organ_address=pd.read_csv('C:/Users/Desktop/Grant_org1.csv')
organ_address['n.org_name']=organ_address.apply(lambda x: x['n.org_name'].replace('"','').lower().strip(),axis=1)

def extract_and_create_researcher_location_nodes_Grant():
 with driver.session() as session:
  query = "MATCH (i:Grant) RETURN i.org_name AS org_name, i.pi_name AS pi_name"
  results = session.run(query)
  for Grant in results:
   Name= Grant['pi_name']
   Aff_name= str(Grant['org_name'])
   if (Name, Aff_name) not in current_names:
     data=organ_address[organ_address['n.org_name'] == Grant['org_name'].lower().strip()]
     '''
      or
      from geopy.geocoders import Nominatim
      geolocator = Nominatim(user_agent="GetLoc", timeout=30)  # You can adjust the timeout value as needed
      location = geolocator.geocode('NIH')
      if location:
              address = location.address.split(", ")
              data={}
              data['Country'] = address[-1]
              data['zip_code'] = address[-2]
              data['state']  = address[-3]  # Massachusetts (state)
              data['County'] = address[-4]  # Massachusetts (state)
              data['city'] = address[-5]  # Boston (city)
     '''
     if len(data) >0 :
       event_properties = {
        'Name': Name,
        'Contact_info': '',
        'Aff_name': Aff_name,
        'Aff_Zip':  data.iloc[0]['zip_code']
       }
       create_Researcher_node(event_properties)
       A,B,C=data.iloc[0]['Country'], data.iloc[0]['state'], data.iloc[0]['County']
       if  type(A)==str and type(B)==str and type(C) == str:
        event_properties = {
        'Aff_country': A ,
        'Aff_state': B,
        'Aff_county': C,
        'Aff_city': data.iloc[0]['city'],
        'Aff_FIPS': find_fip(data.iloc[0]['city'], data.iloc[0]['state'] )[0]
        }
        create_Location_node(event_properties, Name, Aff_name)
     else:
         event_properties = {'Name': Name,'Contact_info': '', 'Aff_name': Aff_name, 'Aff_Zip': ''    }
         create_Researcher_node(event_properties)
     current_names[(Name, Aff_name)] =1

current_names={}
extract_and_create_researcher_location_nodes_Grant()

In [ ]:
def create_relationship_between_nodes_1(node1_label, node2_label, relationship_type):
    with driver.session() as session:
        query = (
            f"MATCH (n1:{node1_label}), (n2:{node2_label}) "
            f"WHERE n1.Name = n2.OfficialName AND n1.Aff_name = n2.OfficialAffiliation "
            f"CREATE (n1)-[:{relationship_type}]->(n2)"
        )
        session.run(query)
def create_relationship_between_nodes_2(node1_label, node2_label, relationship_type):
    with driver.session() as session:
        query = (
        f"MATCH (n1:{node1_label}), (n2:{node2_label}) "
        f"WHERE n1.Name = n2.pi_name and  n1.Aff_name= n2.org_name "
       f"CREATE (n1)-[:{relationship_type}]->(n2)"
        )
        session.run(query)
def create_relationship_between_nodes_3(node1_label, node2_label, relationship_type):
  with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        query = (
        f"MATCH (n1:{node1_label}), (n2:{node2_label}) "
        f"WHERE n1.Name = n2.fullName and  n1.Aff_name= n2.ar_affiliation "
       f"CREATE (n1)-[:{relationship_type}]->(n2)"
        )
        session.run(query)

create_relationship_between_nodes_1('Researcher','ClinicalTrial',  "Involves_Researcher_ct")
create_relationship_between_nodes_2('Researcher','Grant',  "Involves_Researcher_g")

create_relationship_between_nodes_3('Researcher','Pubmed',  "Involves_Researcher_p")

In [ ]:
#faster way
def find_coauthors(df, name, affiliation, title):
    coauthors_dict = {}
    connection = {}
    # Group rows by title
    grouped = df.groupby(title)
    for title_value, group in grouped:
        author_affiliation_set = set()

        for _, row in group.iterrows():
            author = (row[name], row[affiliation])

            # Skip if author name or affiliation is empty
            if not all(author):
                continue

            # If the author is not already seen for this title
            if author not in author_affiliation_set:
                author_affiliation_set.add(author)
            else:
                # This author has already been considered for this title
                continue

            # Add author to coauthors_dict if not already present
            if author not in coauthors_dict:
                coauthors_dict[author] = []

            # Find co-authors within the same title group
            coauthors = group[(group[name] != row[name]) | (group[affiliation] != row[affiliation])]
            for _, coauthor_row in coauthors.iterrows():
                coauthor = (coauthor_row[name], coauthor_row[affiliation])

                # Skip if coauthor name or affiliation is empty
                if not all(coauthor):
                    continue

                # Avoid duplicates
                if coauthor not in coauthors_dict[author]:
                    coauthors_dict[author].append(coauthor)

                # Update connection count
                connection[(author, coauthor)] = connection.get((author, coauthor), 0) + 1

    return coauthors_dict, connection
import time
from collections import defaultdict

def merge_dicts(d1, d2):
    combined_dict = defaultdict(set)
    for key, values in d1.items():
        combined_dict[key].update(values)
    for key, values in d2.items():
        combined_dict[key].update(values)
    return combined_dict



def sum_dicts(dict1, dict2):
    result_dict = {}
    all_keys = set(dict1.keys()) | set(dict2.keys())
    for key in all_keys:
        value1 = dict1.get(key, 0)
        value2 = dict2.get(key, 0)
        result_dict[key] = value1 + value2
    return result_dict
def extract_and_create_researcher_connection():
 with driver.session() as session:
  query = "MATCH (i:ClinicalTrial) RETURN i.OfficialName AS OfficialName, i.OfficialAffiliation AS OfficialAffiliation, i.NCTId as NCTId "
  results = session.run(query)
  data = [record.data() for record in results]  # Convert result to a list of dictionaries
  ClinicalTrial = pd.DataFrame(data)
  print("--- start ---")
  start_time = time.time()
  a,a1=find_coauthors(ClinicalTrial,'OfficialName','OfficialAffiliation','NCTId')
  print("--- %s seconds ---" % (time.time() - start_time))
  ######   Grant
  query = "MATCH (i:Grant) RETURN i.pi_name AS pi_name, i.org_name AS org_name, i.application_id as application_id "
  results = session.run(query)
  data = [record.data() for record in results]  # Convert result to a list of dictionaries
  Grant = pd.DataFrame(data)
  start_time = time.time()
  b,b1=find_coauthors(Grant,'pi_name','org_name','application_id')
  print("--- %s seconds ---" % (time.time() - start_time))
  ##################### pubmed
  query = "MATCH (i:Pubmed) RETURN i.fullName AS fullName, i.ar_affiliation AS affiliation, i.pubmed_id as pubmed_id "
  results = session.run(query)
  data = [record.data() for record in results]  # Convert result to a list of dictionaries
  Pubmed = pd.DataFrame(data)
  start_time = time.time()
  c,c1=find_coauthors(Pubmed,'fullName','affiliation','pubmed_id')
  print("--- %s seconds ---" % (time.time() - start_time))
  ###################
  combined_dict = merge_dicts(a, b)
  combined_dict = merge_dicts(combined_dict, c)
  ###########
  combined_dict1 = sum_dicts(a1, b1)
  result_dict1 = sum_dicts(combined_dict1, c1)
  for key in result_dict1:
    result_dict1[key] = result_dict1[key] %1000
  ###########
  result_dict = {key: list(values) for key, values in combined_dict.items()}
  return combined_dict, result_dict1, a,a1,b,b1,c,c1
result_dict, result_dict1, a,a1,b,b1,c,c1=extract_and_create_researcher_connection()


--- start ---
--- 0.78605055809021 seconds ---
--- 5.310334920883179 seconds ---
--- 119.05593013763428 seconds ---


In [ ]:
n=0
for i in result_dict1:
    n+=1
print(n)

778210


In [ ]:
gg=Grant.groupby('application_id')
bb=pd.DataFrame(gg)

In [ ]:
for i in range(10000):
    if bb[1][i].shape[0] >3:
         display(i,bb[1][i])
         break

542

,pi_name,org_name,application_id
13317,"PARTIDA-SANCHEZ, SANTIAGO",RESEARCH INST NATIONWIDE CHILDREN'S HOSP,10445615
13318,"AMER, AMAL O",RESEARCH INST NATIONWIDE CHILDREN'S HOSP,10445615
13319,"HALL-STOODLEY, LUANNE",RESEARCH INST NATIONWIDE CHILDREN'S HOSP,10445615
13320,"KOPP, BENJAMIN T.",RESEARCH INST NATIONWIDE CHILDREN'S HOSP,10445615


In [ ]:
for i in coauthors_dict:
    if len(coauthors_dict[i])>0: print(coauthors_dict[i])

In [ ]:
driver = GraphDatabase.driver(uri, auth=(username, password))

def remove_collaborated_with_edges():
    with driver.session() as session:
        session.run("MATCH ()-[r:Collaborated_With]->() DELETE r")

remove_collaborated_with_edges()
driver.close()

In [ ]:
import time
import threading

# Function to prevent sleep
def prevent_sleep():
    while True:
        time.sleep(60)  # Keep running indefinitely

In [ ]:
def create_relationship_between_Researchers(n1_Name, n1_Aff_name, n2_Name, n2_Aff_name, num_colab):
  with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        # Ensure no duplicate relationships are created
        create_query = (
            "MATCH (n1:Researcher {Name: $n1_Name, Aff_name: $n1_Aff_name}), "
            "(n2:Researcher {Name: $n2_Name, Aff_name: $n2_Aff_name}) "
            "MERGE (n1)-[r:Collaborated_With]-(n2) "
            "ON CREATE SET r.num_colab = $num_colab "
            "ON MATCH SET r.num_colab = r.num_colab + $num_colab"
        )
        session.run(create_query, n1_Name=n1_Name, n1_Aff_name=n1_Aff_name, n2_Name=n2_Name, n2_Aff_name=n2_Aff_name, num_colab=num_colab)
#61050--- 130,000
#204485----290485
n=0
for key in result_dict1:
   if n> 277633 and n <290485:
    create_relationship_between_Researchers(key[0][0], key[0][1], key[1][0], key[1][1], result_dict1[key])
   n+=1

In [ ]:
n=0
for key in result_dict1:
    n+=1
print(n)

778210


# US MAp

In [ ]:
!pip install python-louvain
import networkx as nx
layout_algorithms = [
    nx.circular_layout,
    nx.random_layout,
    nx.spring_layout,
    nx.spectral_layout,
    nx.shell_layout,
    nx.bipartite_layout,
    nx.planar_layout,
    nx.fruchterman_reingold_layout,
    nx.kamada_kawai_layout
]

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Using cached python-louvain-0.16.tar.gz (204 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.6 MB 10.3 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.6 MB 12.6 MB/s eta 0:00:01
   --------------------------- ------------ 8.7/12.6 MB 14.2 MB/s eta 0:00:01
   -------------

In [ ]:
#faster way
import pandas as pd
def find_coauthors(df, name, affiliation, title):
    coauthors_dict = {}
    connection = {}
    # Group rows by title
    grouped = df.groupby(title)
    for title_value, group in grouped:
        author_affiliation_set = set()

        for _, row in group.iterrows():
            author = (row[name], row[affiliation])

            # Skip if author name or affiliation is empty
            if not all(author):
                continue

            # If the author is not already seen for this title
            if author not in author_affiliation_set:
                author_affiliation_set.add(author)
            else:
                # This author has already been considered for this title
                continue

            # Add author to coauthors_dict if not already present
            if author not in coauthors_dict:
                coauthors_dict[author] = []

            # Find co-authors within the same title group
            coauthors = group[(group[name] != row[name]) | (group[affiliation] != row[affiliation])]
            for _, coauthor_row in coauthors.iterrows():
                coauthor = (coauthor_row[name], coauthor_row[affiliation])

                # Skip if coauthor name or affiliation is empty
                if not all(coauthor):
                    continue

                # Avoid duplicates
                if coauthor not in coauthors_dict[author]:
                    coauthors_dict[author].append(coauthor)

                # Update connection count
                connection[(author, coauthor)] = connection.get((author, coauthor), 0) + 1

    return coauthors_dict, connection
import time
from collections import defaultdict

def merge_dicts(d1, d2):
    combined_dict = defaultdict(set)
    for key, values in d1.items():
        combined_dict[key].update(values)
    for key, values in d2.items():
        combined_dict[key].update(values)
    return combined_dict



def sum_dicts(dict1, dict2):
    result_dict = {}
    all_keys = set(dict1.keys()) | set(dict2.keys())
    for key in all_keys:
        value1 = dict1.get(key, 0)
        value2 = dict2.get(key, 0)
        result_dict[key] = value1 + value2
    return result_dict
def extract_and_create_researcher_connection(disease):
 with GraphDatabase.driver(uri, auth=(username, password)) as driver:
  with driver.session() as session:
   query = "MATCH (l:Location)--(r:Researcher)--(i:ClinicalTrial)--(g:Gard) where g.GARDname= $disease and COALESCE(size(l.Aff_FIPS), 0) > 3    RETURN i.OfficialName AS OfficialName, i.OfficialAffiliation AS OfficialAffiliation, i.NCTId as NCTId, l.Aff_FIPS as FIPS "
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   ClinicalTrial = pd.DataFrame(data)
   print("--- start ---")
   start_time = time.time()
   if ClinicalTrial.shape[0]>0: a,a1=find_coauthors(ClinicalTrial,'OfficialName','OfficialAffiliation','NCTId')
   else: a,a1=pd.DataFrame(),pd.DataFrame()
   print("--- %s seconds ---" % (time.time() - start_time))
   ######   Grant
   query = "MATCH (l:Location)--(r:Researcher)--(i:Grant)--(g:Gard) where g.GARDname= $disease and COALESCE(size(l.Aff_FIPS), 0) > 3    RETURN i.pi_name AS pi_name, i.org_name AS org_name, i.application_id as application_id , l.Aff_FIPS as FIPS "
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   Grant = pd.DataFrame(data)
   start_time = time.time()
   if Grant.shape[0]>0:b,b1=find_coauthors(Grant,'pi_name','org_name','application_id')
   else: b,b1=pd.DataFrame(),pd.DataFrame()
   print("--- %s seconds ---" % (time.time() - start_time))
   ##################### pubmed
   query = "MATCH (l:Location)--(r:Researcher)--(i:Pubmed)--(g:Gard) where g.GARDname= $disease and COALESCE(size(l.Aff_FIPS), 0) > 3   RETURN i.fullName AS fullName, i.ar_affiliation AS affiliation, i.pubmed_id as pubmed_id, l.Aff_FIPS as FIPS "
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   Pubmed = pd.DataFrame(data)
   start_time = time.time()
   if Pubmed.shape[0]>0:c,c1=find_coauthors(Pubmed,'fullName','affiliation','pubmed_id')
   else: c,c1=pd.DataFrame(),pd.DataFrame()
   print("--- %s seconds ---" % (time.time() - start_time))
   ###################
   combined_dict = merge_dicts(a, b)
   combined_dict = merge_dicts(combined_dict, c)
   ###########
   combined_dict1 = sum_dicts(a1, b1)
   result_dict1 = sum_dicts(combined_dict1, c1)
   for key in result_dict1:
    result_dict1[key] = result_dict1[key] %1000
   ###########
   result_dict = {key: list(values) for key, values in combined_dict.items()}
   return combined_dict, result_dict1, a,a1,b,b1,c,c1
disease= 'Cystic fibrosis'
result_dict, result_dict1, a,a1,b,b1,c,c1=extract_and_create_researcher_connection(disease)
print(len(result_dict))

--- start ---
--- 0.3788144588470459 seconds ---
--- 4.489328145980835 seconds ---
--- 2.0419180393218994 seconds ---
3455


In [ ]:
import networkx as nx
import community
from community import community_louvain
import matplotlib.pyplot as plt
# Create a directed graph
G = nx.Graph()
# Add nodes and edges from the dictionary
for nodes, edges in result_dict.items():
    G.add_node(nodes)
    for edge in edges:
        G.add_node(edge)
        G.add_edge(nodes, edge)

# Detect communities using Louvain method
partition = community_louvain.best_partition(G)
data_table = pd.DataFrame(partition.items(), columns=['name', 'Community'])
data_table['network']=data_table.apply(lambda x: result_dict[x['name']] ,axis=1)
index_community_detection= list(set(data_table[data_table['network']!=set()]['Community'].values))

In [ ]:
def merge_community_detection_and_researchers_info(disease):
 with GraphDatabase.driver(uri, auth=(username, password)) as driver:
  with driver.session() as session:
   query = "MATCH (l:Location)--(r:Researcher)--(i:ClinicalTrial)--(g:Gard) WHERE g.GARDname= $disease and COALESCE(size(l.Aff_FIPS), 0) > 3 RETURN i.OfficialName as OfficialName , i.OfficialAffiliation as OfficialAffiliation, i.OfficialTitle as OfficialTitle  ,i.Interventions as concatenatedInterventions, i.BriefSummary as BriefSummary, l.Aff_FIPS as FIPS"
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   if data==[]:  ClinicalTrial = pd.DataFrame(columns=["OfficialName", "OfficialAffiliation", "OfficialTitle", "concatenatedInterventions", "BriefSummary,FIPS"])
   else: ClinicalTrial = pd.DataFrame(data)
   ######   Grant
   query = "MATCH (l:Location)--(r:Researcher)--(i:Grant)--(g:Gard) WHERE g.GARDname= $disease and COALESCE(size(l.Aff_FIPS), 0) > 3 RETURN i.pi_name as pi_name,i.org_name as org_name,i.title as title ,i.terms as terms,i.abstract as abstract, l.Aff_FIPS as FIPS"
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries

   if data==[]:  Grant = pd.DataFrame(columns=["pi_name", "org_name", "title", "terms", "abstract","FIPS"])
   else: Grant = pd.DataFrame(data)
   ##################### pubmed
   query = "MATCH (l:Location)--(r:Researcher)--(i:Pubmed)--(g:Gard) WHERE g.GARDname= $disease and COALESCE(size(l.Aff_FIPS), 0) > 3 RETURN i.fullName as fullName, i.ar_affiliation as affiliation, i.title as title, i.concatenatedKeywords as concatenatedKeywords,i.abstractText as abstractText, l.Aff_FIPS as FIPS "
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries

   if data==[]:  Pubmed = pd.DataFrame(columns=["fullName", "affiliation", "title", "concatenatedKeywords","abstractText", "FIPS"])
   else: Pubmed = pd.DataFrame(data)
   ################
   if ClinicalTrial.shape[0]>0:  ClinicalTrial['name'] = ClinicalTrial.apply(lambda row: (row['OfficialName'], row['OfficialAffiliation']), axis=1)
   else: ClinicalTrial['name'] = None
   if Grant.shape[0]>0:          Grant['name'] = Grant.apply(lambda row: (row['pi_name'], row['org_name']), axis=1)
   else: Grant['name'] = None
   if Pubmed.shape[0]>0:         Pubmed['name'] = Pubmed.apply(lambda row: (row['fullName'], row['affiliation']), axis=1)
   else: Pubmed['name'] = None

   ClinicalTrial['BriefSummary']=ClinicalTrial['OfficialTitle'].fillna("").astype(str)+' '+ ClinicalTrial['BriefSummary'].fillna("").astype(str)   #+ClinicalTrial['concatenatedInterventions'].fillna("").astype(str)+
   Grant['BriefSummary']=Grant['title'].fillna("").astype(str)+' '+ Grant['abstract'].fillna("").astype(str)                   #+Grant['terms'].fillna("").astype(str)
   Pubmed['abstractText']=Pubmed['title'].fillna("").astype(str)+' '+ Pubmed['abstractText'].fillna("").astype(str)    #+Pubmed['concatenatedKeywords'].fillna("").astype(str)

   ClinicalTrial['Terms']=ClinicalTrial['concatenatedInterventions']
   Grant['Terms']=Grant['terms']
   Pubmed['Terms']=Pubmed['concatenatedKeywords']
   #df1=ClinicalTrial[['i.OfficialName',  'c.BriefSummary', 'i.OfficialAffiliation','name']]
   df1=ClinicalTrial[['OfficialName',  'BriefSummary','OfficialAffiliation', 'name','Terms',"FIPS"]]
   df2=Grant[['pi_name', 'abstract', 'org_name','name','Terms',"FIPS"]]
   df3=Pubmed[['fullName',  'abstractText', 'affiliation','name','Terms',"FIPS"]]
   # Rename columns to avoid conflicts
   df1.columns = ['PI','Abstract', 'affiliation','name','Terms',"FIPS"]
   df2.columns = ['PI','Abstract', 'affiliation','name','Terms',"FIPS"]
   df3.columns = ['PI','Abstract', 'affiliation','name','Terms',"FIPS"]
   #df2['Location']=  df2['locationCity'] +' , '+df2['Location']
   result_df = pd.concat([df1  , df2, df3])
   #result_df = pd.concat([df1,  df3])
   result_df=result_df.drop_duplicates()
   result_df.index= range(result_df.shape[0])
   #################################################### merging with c

   merged_data1 = pd.merge(result_df,data_table , on='name', how='outer')
   concatenated_data = merged_data1.drop_duplicates(subset=['name'])
   concatenated_data.reset_index(drop=True, inplace=True)
   #concatenated_data=concatenated_data.drop(['name'],axis=1)
   return concatenated_data
   '''
   #return result_df
   '''
concatenated_data=merge_community_detection_and_researchers_info(disease)

In [ ]:
concatenated_data

,PI,Abstract,affiliation,name,Terms,FIPS,Community,network
0,Anthony Bier,"A Prospective, Randomized, Double-blinded, Pla...",Nutrasource Pharmaceutical and Nutraceutical S...,"(Anthony Bier, Nutrasource Pharmaceutical and ...","[Placebo, Probiotic]",12086,280.0,{}
1,Mina Pastagia,"A Phase 2, Multi-Center, Double-Blind, Randomi...","Armata Pharmaceuticals, Inc.","(Mina Pastagia, Armata Pharmaceuticals, Inc.)","[Placebo, AP-PA02]",53033,253.0,{}
2,Mary C. Drinane,Relationship Between the Development of Impair...,Dartmouth-Hitchcock Medical Center,"(Mary C. Drinane, Dartmouth-Hitchcock Medical ...",[Fibroscan],33009,271.0,{}
3,H. Frank Farmer,An Open-Label Phase 1 Study to Examine the Eff...,"Covance CRU, Inc.","(H. Frank Farmer, Covance CRU, Inc.)",[VX-770],12127,91.0,{}
4,Lisa Guay-Woodford,Core A: The Hepato/Renal Fibrocystic Diseases ...,Childrens National Health System,"(Lisa Guay-Woodford, Childrens National Health...",[],11001,118.0,{}
...,...,...,...,...,...,...,...,...
3537,Greenberg J,<h4>Rationale</h4>Monitoring clinical disease...,"Division of Respiratory Diseases, Boston Child...","(Greenberg J, Division of Respiratory Diseases...",[],25025,1769.0,"{(Walt DR, Department of Chemistry, Tufts Univ..."
3538,Diggle SP,<i>Pseudomonas aeruginosa</i> is an opportuni...,"Center for Microbial Dynamics and Infection, S...","(Diggle SP, Center for Microbial Dynamics and ...","[evolution, antibiotic resistance, population ...",13121,2072.0,"{(Vanderwoude J, Center for Microbial Dynamics..."
3539,Leal J,Pseudomonas aeruginosa is the predominant pat...,Division of Molecular Pharmaceutics and Drug D...,"(Leal J, Division of Molecular Pharmaceutics a...","[cystic fibrosis, tobramycin, biofilm, pegylat...",48453,1912.0,"{(Smyth HDC, Division of Molecular Pharmaceuti..."
3540,Strubberg AM,Increased intestinal permeability is a manife...,"Dalton Cardiovascular Research Center, Univers...","(Strubberg AM, Dalton Cardiovascular Research ...","[enteroid, cdc42, cystic fibrosis, permeabilit...",29041,2110.0,"{(Woode RA, Dalton Cardiovascular Research Cen..."


In [ ]:
fip_finding=pd.read_csv('C:/Users/Desktop/Social network all works/work rdas/uscities.csv')
fip_finding_dict=dict()
lat_finding_dict=dict()
lon_finding_dict=dict()
lon_lag_fip_info=dict()
for i in fip_finding.index:
     fip_finding_dict[fip_finding['city'][i].lower()]= fip_finding['county_fips'][i]
     lat_finding_dict[fip_finding['city'][i].lower()]= fip_finding['lat'][i]
     lon_finding_dict[fip_finding['city'][i].lower()]= fip_finding['lng'][i]
     lon_lag_fip_info[( round(fip_finding['lat'][i]) ,round(fip_finding['lng'][i]) )]=fip_finding['city'][i]
for i in concatenated_data.index:
 try:
  concatenated_data.at[i,'lat']= fip_finding[fip_finding['county_fips'].astype(str)== str(concatenated_data['FIPS'][i]) ]['lat'].iloc[0]
  concatenated_data.at[i,'lng']= fip_finding[fip_finding['county_fips'].astype(str)== str(concatenated_data['FIPS'][i]) ]['lng'].iloc[0]
 except:
     pass

In [ ]:
concatenated_data ['fip']=concatenated_data ['FIPS']
concatenated_data= concatenated_data.drop('FIPS' , axis=1)
concatenated_data

,PI,Abstract,affiliation,name,Terms,Community,network,lat,lng,fip
0,Anthony Bier,"A Prospective, Randomized, Double-blinded, Pla...",Nutrasource Pharmaceutical and Nutraceutical S...,"(Anthony Bier, Nutrasource Pharmaceutical and ...","[Placebo, Probiotic]",280.0,{},25.7840,-80.2101,12086
1,Mina Pastagia,"A Phase 2, Multi-Center, Double-Blind, Randomi...","Armata Pharmaceuticals, Inc.","(Mina Pastagia, Armata Pharmaceuticals, Inc.)","[Placebo, AP-PA02]",253.0,{},47.6211,-122.3244,53033
2,Mary C. Drinane,Relationship Between the Development of Impair...,Dartmouth-Hitchcock Medical Center,"(Mary C. Drinane, Dartmouth-Hitchcock Medical ...",[Fibroscan],271.0,{},43.6353,-72.2531,33009
3,H. Frank Farmer,An Open-Label Phase 1 Study to Examine the Eff...,"Covance CRU, Inc.","(H. Frank Farmer, Covance CRU, Inc.)",[VX-770],91.0,{},28.9050,-81.2137,12127
4,Lisa Guay-Woodford,Core A: The Hepato/Renal Fibrocystic Diseases ...,Childrens National Health System,"(Lisa Guay-Woodford, Childrens National Health...",[],118.0,{},38.9047,-77.0163,11001
...,...,...,...,...,...,...,...,...,...,...
3537,Greenberg J,<h4>Rationale</h4>Monitoring clinical disease...,"Division of Respiratory Diseases, Boston Child...","(Greenberg J, Division of Respiratory Diseases...",[],1769.0,"{(Walt DR, Department of Chemistry, Tufts Univ...",42.3188,-71.0852,25025
3538,Diggle SP,<i>Pseudomonas aeruginosa</i> is an opportuni...,"Center for Microbial Dynamics and Infection, S...","(Diggle SP, Center for Microbial Dynamics and ...","[evolution, antibiotic resistance, population ...",2072.0,"{(Vanderwoude J, Center for Microbial Dynamics...",33.7628,-84.4220,13121
3539,Leal J,Pseudomonas aeruginosa is the predominant pat...,Division of Molecular Pharmaceutics and Drug D...,"(Leal J, Division of Molecular Pharmaceutics a...","[cystic fibrosis, tobramycin, biofilm, pegylat...",1912.0,"{(Smyth HDC, Division of Molecular Pharmaceuti...",30.3005,-97.7522,48453
3540,Strubberg AM,Increased intestinal permeability is a manife...,"Dalton Cardiovascular Research Center, Univers...","(Strubberg AM, Dalton Cardiovascular Research ...","[enteroid, cdc42, cystic fibrosis, permeabilit...",2110.0,"{(Woode RA, Dalton Cardiovascular Research Cen...",39.4233,-92.8025,29041


In [ ]:
concatenated_data.to_csv('concatenated_data.csv')

In [ ]:
concatenated_data1 = concatenated_data  # Initialize concatenated_data1
Node = list(set(concatenated_data1['fip'].values))  # Extract unique 'fip' values as nodes
Edge = []

# Iterate through each row in concatenated_data1
for i in concatenated_data1.index:
    # Ensure the 'network' value is iterable and not NaN
    if isinstance(concatenated_data1['network'][i], (list, str)):
        # Iterate through each element in the 'network' column for the current row
        for j in concatenated_data1['network'][i]:
            # Filter data where 'name' matches the current network element (j)
            filtered_data = concatenated_data1[concatenated_data1['name'] == j]

            # If filtered data is not empty, create an edge
            if not filtered_data.empty:
                value = filtered_data['fip'].iloc[0]  # Get the 'fip' of the first matching row
                edge = (concatenated_data1['fip'][i], value)  # Create the edge (tuple)
                Edge.append(edge)  # Add the edge to the list
    else:
        # If 'network' value is not iterable (e.g., NaN or not a list/str), skip it
        continue

# Remove duplicate edges (if any) by converting the list of edges to a set and back to a list
Edge = list(set(Edge))

In [ ]:
import networkx as nx
from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.
#G = nx.Graph()
G=nx.DiGraph()
G.add_nodes_from(Node)
G.add_edges_from(Edge)
#nx.info(G)

In [ ]:
Node_utility=Node
Edge_utility=[]
Fips_utlity=Node
Lat_utlity=[]
Log_utlity=[]
n=0
while n < len(Node_utility):
      Lat_utlity.append(  concatenated_data1[concatenated_data1['fip']==Node[n]]['lat'].iloc[0] )
      Log_utlity.append(  concatenated_data1[concatenated_data1['fip']==Node[n]]['lng'].iloc[0] )
      n+=1
      #Node_utility.append(Utility_County[0][i])

G_utility=nx.DiGraph()
G_utility.add_nodes_from(Node_utility)
G_utility.add_edges_from(Edge_utility)

In [ ]:
import numpy as np
Fips_latlong_utility=pd.DataFrame()
Fips_latlong_utility['Utility Number']=Node_utility
Fips_latlong_utility['fips']=Node_utility
Fips_latlong_utility['Latitude']=Lat_utlity
Fips_latlong_utility['Longitude']=Log_utlity


data_G=pd.DataFrame()
data_G['Node']=Node
d = nx.degree(G)
data_G['degree']=d

data_G_utility=pd.DataFrame()
data_G_utility['Node']=Fips_latlong_utility['Utility Number']
data_G_utility['degree']=''
for i in range(data_G_utility.shape[0]):
    if Fips_latlong_utility['fips'][i] in data_G['Node'].values:
        data_G_utility.at[i,'degree'] = data_G.loc[data_G['Node'] == Fips_latlong_utility['fips'][i]]['degree'].item()
    else:
        data_G_utility.at[i,'degree'] = 1

In [ ]:
for i in range(data_G_utility.shape[0]):
     data_G_utility.at[i,'degree']=data_G_utility['degree'][i][1]
List=np.array(data_G_utility['degree'].astype(float))

In [ ]:
!pip install setuptools
!pip install numpy cython matplotlib


Defaulting to user installation because normal site-packages is not writeable
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
Defaulting to user installation because normal site-packages is not writeable
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 22.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------  7.9/8.0 MB 44.6 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 29.0 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 26.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install -c conda-forge basemap

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'conda-forge'


In [ ]:
from mpl_toolkits.basemap import Basemap as Basemap
plt.figure(figsize = (20,9))
m = Basemap(
         #projection='merc',   'mill',  'cyl'
         llcrnrlon=-130,    #-180,  left
         llcrnrlat=25,         #   10,  south
         urcrnrlon=-50,   #-50, right
         urcrnrlat=50,    #70,  North
         lat_ts=0,
         resolution='l',     #   c - crude,  l - low,  h - high,  f - full
         suppress_ticks=True)

mx, my = m(pd.to_numeric(fip_finding['lng'].astype(str).str.replace('–', '-', regex=True), downcast='float').values,  pd.to_numeric(fip_finding['lat'], downcast='float').values)

############ Counties
#mx, my = m(pd.to_numeric(fip_finding['lng'].str.replace('–', '-', regex=True), downcast='float').values,  pd.to_numeric(fip_finding['lat'], downcast='float').values)
pos = {}

fip_finding['county_fips'] = fip_finding['county_fips'].astype(str)
fip_finding['county_fips'] = fip_finding['county_fips'].apply(lambda x: '{0:0>5}'.format(x))

for count, elem in enumerate (fip_finding['county_fips'].values):
      pos[elem] = (mx[count], my[count])


#pos=nx.spring_layout(G)
d = nx.degree(G)
d = [(d[node]+1) * 25 for node in G.nodes()]
nx.draw_networkx_nodes(G , pos = pos, node_color = 'r', alpha = 0.8, node_size=d) #node_size = 4)
nx.draw_networkx_edges(G , pos = pos, edge_color='g',alpha=0.2, arrows = False, width=2)



####### Utilities
#mx1, my1 = m(pd.to_numeric(Fips_latlong_utility['Longitude'].str.replace('–', '-', regex=True), downcast='float').values,  pd.to_numeric(Fips_latlong_utility['Latitude'], downcast='float').values)
pos1 = {}
#for count, elem in enumerate (Fips_latlong_utility['Utility Number'].values):
#       pos1[elem] = (mx[count], my[count])
n=0
while n<len(Node_utility):
  pos1[Node_utility[n]] = (Log_utlity[n],Lat_utlity[n])
  n+=1

dd = List*1
nx.draw_networkx_nodes(G_utility , pos = pos1,node_shape ='s' , node_color = 'k', alpha = 0.8,node_size=1) #node_size = 4,   node_shape  'so^>v<dph8'
#filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')


m.drawcountries(linewidth = 3)
m.drawstates(linewidth = 0.2)
m.drawcoastlines(linewidth=3)
#m.drawcounties(color='darkred')
#m.bluemarble()
#m.etopo()

plt.tight_layout()
#plt.savefig("/content/drive/Shareddrives/MY Files/Utility-based /map_1.png", format = "png", dpi = 300)
plt.show()

In [ ]:
!pip install python-louvain

In [ ]:
G=nx.Graph()
G.add_nodes_from(Node)
G.add_edges_from(Edge)

###########################################################  Cliques ###################################
cl=nx.enumerate_all_cliques(G)
#print last 10 cliques
print([l for l in cl][-10:])
#"5 Largest Cliques"
print("5 Largest Cliques",sorted([l for l in nx.find_cliques(G)],key=lambda x: len(x),reverse=True)[:5])

##################################################### PAGERANK and HITS  ###################################
pg_rank=sorted([l for l in nx.pagerank(G).items()],key=lambda x: x[1],reverse=True)
print("Top 10 county By Pagerank",pg_rank[:10])

hubs,authorities=nx.hits(G)
hubs=sorted([l for l in hubs.items()],key=lambda x: x[1],reverse=True)
authorities=sorted([l for l in authorities.items()],key=lambda x: x[1],reverse=True)
print("Top 10 Biggest Hubs",hubs[:10])
print("\nTop 10 Biggest Authorities",authorities[:10])


In [ ]:
############################  Communty Detection  ######################
import community #requires separate install -  pip install python-louvain
from community import community_louvain
import warnings

plt.figure(figsize=(20,9))
#pos = nx.spring_layout(Gc,iterations=50,k=2)
partition = community_louvain.best_partition(G)
values = [partition.get(node) for node in G.nodes()]


#####################################################################


plt.axis("off")
plt.title('Social Network'.format(len(set(partition.values()))))
#nx.draw_networkx(Gc, pos = pos, cmap = plt.get_cmap("jet"), node_color = values, node_size = 1, with_labels = False)


nx.draw_networkx_nodes(G , pos = pos, cmap = plt.get_cmap("jet"), node_color = values, alpha = 0.8, node_size=50) #node_size = 4)
nx.draw_networkx_edges(G , pos = pos, edge_color='k',alpha=0.2, arrows = False,width=5)


m.drawcountries(linewidth = 3)
m.drawstates(linewidth = 0.2)
m.drawcoastlines(linewidth=3)
#m.drawcounties(color='darkred')
#m.bluemarble()
#m.etopo()

plt.tight_layout()
plt.show()

In [ ]:
L = nx.laplacian_matrix(G)
eig_values, eig_vectors = np.linalg.eigh(L.todense()) # Eigen values sorted from smallest to biggest
f = eig_vectors[:,1] # use the second smallest Eigen vector for spectral clustering
labels = np.ravel(np.sign(f)) # decides the label of each node


#####################################################################

fig = plt.figure(figsize=(9,6))
plt.axis("off")
#nx.draw_networkx(Gc, pos,node_size=45, cmap = plt.get_cmap("jet"), node_color=labels, with_labels = False)
nx.draw_networkx_nodes(G , pos = pos, cmap = plt.get_cmap("jet"), node_color = labels, alpha = 0.8, node_size=10) #node_size = 4)
nx.draw_networkx_edges(G , pos = pos, edge_color='k',alpha=0.2, arrows = False)


m.drawcountries(linewidth = 3)
m.drawstates(linewidth = 0.2)
m.drawcoastlines(linewidth=3)
#m.drawcounties(color='darkred')
#m.bluemarble()
#m.etopo()

plt.tight_layout()
plt.show()


# Clustering new

In [ ]:
def merge_community_detection_and_researchers_info(disease):
 with GraphDatabase.driver(uri, auth=(username, password)) as driver:
  with driver.session() as session:
   query = "MATCH (i:ClinicalTrial)--(g:Gard) WHERE g.GARDname= $disease RETURN i.OfficialName as OfficialName , i.OfficialAffiliation as OfficialAffiliation, i.OfficialTitle as OfficialTitle  ,i.Interventions as concatenatedInterventions, i.BriefSummary as BriefSummary"
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   if data==[]:  ClinicalTrial = pd.DataFrame(columns=["OfficialName", "OfficialAffiliation", "OfficialTitle", "concatenatedInterventions", "BriefSummary"])
   else: ClinicalTrial = pd.DataFrame(data)
   ######   Grant
   query = "MATCH (i:Grant)--(g:Gard) WHERE g.GARDname= $disease RETURN i.pi_name as pi_name,i.org_name as org_name,i.title as title ,i.terms as terms,i.abstract as abstract"
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   Grant = pd.DataFrame(data)
   ##################### pubmed
   query = "MATCH (i:Pubmed)--(g:Gard) WHERE g.GARDname= $disease RETURN i.fullName as fullName, i.ar_affiliation as affiliation, i.title as title, i.concatenatedKeywords as concatenatedKeywords,i.abstractText as abstractText "
   results = session.run(query, disease=disease)
   data = [record.data() for record in results]  # Convert result to a list of dictionaries
   Pubmed = pd.DataFrame(data)
   ################
   ClinicalTrial['name'] = ClinicalTrial.apply(lambda row: (row['OfficialName'], row['OfficialAffiliation']), axis=1)
   Grant['name'] = Grant.apply(lambda row: (row['pi_name'], row['org_name']), axis=1)
   Pubmed['name'] = Pubmed.apply(lambda row: (row['fullName'], row['affiliation']), axis=1)

   ClinicalTrial['BriefSummary']=ClinicalTrial['OfficialTitle'].fillna("").astype(str)+' '+ ClinicalTrial['BriefSummary'].fillna("").astype(str)   #+ClinicalTrial['concatenatedInterventions'].fillna("").astype(str)+
   Grant['BriefSummary']=Grant['title'].fillna("").astype(str)+' '+ Grant['abstract'].fillna("").astype(str)                   #+Grant['terms'].fillna("").astype(str)
   Pubmed['abstractText']=Pubmed['title'].fillna("").astype(str)+' '+ Pubmed['abstractText'].fillna("").astype(str)    #+Pubmed['concatenatedKeywords'].fillna("").astype(str)

   ClinicalTrial['Terms']=ClinicalTrial['concatenatedInterventions']
   Grant['Terms']=Grant['terms']
   Pubmed['Terms']=Pubmed['concatenatedKeywords']
   #df1=ClinicalTrial[['i.OfficialName',  'c.BriefSummary', 'i.OfficialAffiliation','name']]
   df1=ClinicalTrial[['OfficialName',  'BriefSummary','OfficialAffiliation', 'name','Terms']]
   df2=Grant[['pi_name', 'abstract', 'org_name','name','Terms']]
   df3=Pubmed[['fullName',  'abstractText', 'affiliation','name','Terms']]
   # Rename columns to avoid conflicts
   df1.columns = ['PI','Abstract', 'affiliation','name','Terms']
   df2.columns = ['PI','Abstract', 'affiliation','name','Terms']
   df3.columns = ['PI','Abstract', 'affiliation','name','Terms']
   #df2['Location']=  df2['locationCity'] +' , '+df2['Location']
   result_df = pd.concat([df1  , df2, df3])
   #result_df = pd.concat([df1,  df3])
   result_df=result_df.drop_duplicates()
   result_df.index= range(result_df.shape[0])
   #################################################### merging with c
   '''
  merged_data1 = pd.merge(result_df,data_table , left_on='name', right_on='Node', how='outer')
  concatenated_data = merged_data1.drop_duplicates(subset=['name'])
  concatenated_data.reset_index(drop=True, inplace=True)
  concatenated_data=concatenated_data.drop(['name'],axis=1)
  return concatenated_data
   '''
   return result_df


In [ ]:
import pandas as pd

def sum_strings(series):
    # Convert all items to string, handling NaNs, and remove duplicates by converting to a set
    return ', '.join(sorted(set(str(item) for item in series if pd.notna(item))))

# Function to clean and flatten terms
def sum_terms(series):
    result = []
    for item in series:
        if isinstance(item, list):  # Check if item is a list
            for subitem in item:
                if isinstance(subitem, str):  # Ensure it's a string
                    subitem_cleaned = subitem.strip('[]')  # Remove surrounding brackets
                    result.extend(subitem_cleaned.split(','))  # Split and add terms
        elif pd.notna(item):  # Handle non-list, non-NaN values
            item_cleaned = str(item).strip('[]')  # Convert to string and remove brackets
            result.extend(item_cleaned.split(','))  # Split and add terms
    # Remove extra whitespace from terms
    return [term.strip() for term in result if term.strip()]


In [ ]:
import ast
from collections import Counter
def three_most_frequent_terms(term_string):
    # Convert the string representation of the list to an actual list
    #term_list = ast.literal_eval(term_string)
    # Count the frequency of each term in the list
    term_counts = Counter(term_string)
    # Get the three most common terms
    most_common_terms = term_counts.most_common(5)

    return ' , '.join([term  for term, count in most_common_terms])

# Example usage
terms = [
    'Tobramycin for Inhalation',
    'Tobramycin solution for inhalation (TOBI), Oral placebo, Oral ciprofloxacin',
    'Tobramycin for Inhalation',
    'Tobramycin for Inhalation',
    'Tobramycin for Inhalation',
    'Tobramycin for Inhalation',
    'Tobramycin for Inhalation',
    'Tobramycin for Inhalation',
    'Tobramycin for Inhalation',
    'Tobramycin solution for inhalation (TOBI), Oral placebo, Oral ciprofloxacin',
    'Tobramycin solution for inhalation (TOBI), Oral placebo, Oral ciprofloxacin']

print(three_most_frequent_terms(terms))
#data['frequent_words']  = data.apply(lambda x:  three_most_frequent_terms(x['Terms'])   , axis=1)

Tobramycin for Inhalation , Tobramycin solution for inhalation (TOBI), Oral placebo, Oral ciprofloxacin


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer

def name_clusters(df, text_column, cluster_column, eps=0.5, min_samples=5, n_components=100):
    # Extract abstracts from the specified column and handle NaN values
    abstracts = df[text_column].fillna("").astype(str).tolist()

    # TF-IDF vectorization with ngram_range (optimize based on your dataset)
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))  # Bi-grams may help for better context
    tfidf_matrix = vectorizer.fit_transform(abstracts)

    # Optionally reduce dimensionality to speed up clustering
    # if n_components:
    #     svd = TruncatedSVD(n_components=n_components, random_state=42)
    #     tfidf_matrix = svd.fit_transform(tfidf_matrix)

    # DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine')  # Using cosine distance as the metric
    clusters = dbscan.fit_predict(tfidf_matrix)

    # Assign cluster labels to the DataFrame
    df[cluster_column] = clusters+2

    return df

disease= 'Cystic fibrosis'  #'Actinic prurigo' #'Non-syndromic polydactyly'   #'Cystic fibrosis'
#diseaseid=B[0] #'GARD0017510'

concatenated_data= merge_community_detection_and_researchers_info(disease)
# Assuming you have a DataFrame `clustering_2`
clustering_ = name_clusters(concatenated_data, 'Abstract', 'Community', eps=0.9, min_samples=10, n_components=100)
# Group by 'Community' and aggregate columns using sum_strings and sum_terms functions
result = clustering_.groupby('Community').agg(
    Abstract=('Abstract', sum_strings),
    Terms=('Terms', sum_terms),
    Size=('Community', 'size')  # Count of occurrences
).reset_index()

data=result

print(len(clustering_['Abstract'].unique()))
print('------------------')
for i in clustering_['Community'].unique():
    ASD= clustering_[clustering_['Community']==i]
    print( ASD['Abstract'].drop_duplicates().shape[0] )

41061
------------------
37643
3091
2
2
1
2
3
1
5
4
7
2
2
2
4
1
1
1
1
1
2
1
1
1
1
2
1
1
3
1
1
1
2
1
4
1
1
1
2
1
1
1
2
1
1
2
2
1
1
1
1
3
2
1
6
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
2
1
1
1
1
2
2
1
1
2
1
1
1
1
1
3
1
1
1
2
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
2
2
1
1
1
1
1
2
2
1
1
1
2
3
2
1
2
1
2
1
1
1
1
3
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
2
2
3
4
1
1
1
1
1
2
1
1
1
1
1
1
1
3
1
2
1
4
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
3
1
1
1
1
2


In [ ]:
def convert_to_list(text):
    # Remove the surrounding square brackets and strip any leading/trailing whitespaces
    text=text.replace('<|end_of_text|>','')
    text = text.strip("[]")
    # Split by commas to get individual elements
    words = text.split(',')
    # Clean each word by stripping extra spaces and quotes
    cleaned_words = [word.strip().strip("'\"")  for word in words   if word.strip().strip("'\"")  !='']
    return cleaned_words
import ast

def str_to_dict(s):
    try:
        # Use ast.literal_eval to safely evaluate the string as a dictionary
        return ast.literal_eval(s)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing string: {e}")
        return None

# Example usage:
str_dict = "{'key1': 'value1', 'key2': 'value2'}"
converted_dict = str_to_dict(str_dict)

In [ ]:
data['frequent_words']  = data.apply(lambda x:  three_most_frequent_terms(x['Terms'])   , axis=1)


In [ ]:
def create_Cluster_node(properties, disease):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            # Query to create a Cluster node and link it to the Gard node via a relationship
            query = (
                "MATCH (g:Gard {GARDname: $disease}) "  # Match the Gard node with GARDname = $disease
                "CREATE (Cl:Cluster {"
                'Cluster_ID: $Cluster_ID,'
                'Cluster_Size: $Cluster_Size,'
                'Evidence: $Evidence,'
                'Key_trems: $Key_trems'
                "}) "
                "CREATE (Cl)-[:Cluster_to_GARD]->(g)"  # Create the relationship between Gard and Cluster
            )
            session.run(query, **properties, disease=disease)

            # Create Expertise nodes for the current cluster if applicable
            #if properties['Llama_3_n']:  # If 'Key_trems' exists in properties
            #      dic_expertise=str_to_dict(properties['Llama_3_n'])# Convert to list of expertise terms
            #      for sentence in dic_expertise:
            #           if (sentence!= '') and (sentence.lower() != disease.lower()) and (dic_expertise[sentence]!=''):
            #                          create_expertise_node(session, sentence, dic_expertise[sentence],properties['Cluster_ID'], disease)



def create_expertise_node(session, sentence,explan, cluster_id, disease):
    # Create a new Expertise node with a relationship to the Cluster
    query = (
        "MERGE (e:Expertise {Summarized_expertise: $sentence, expertise_explanation:$explan}) "
        "WITH e "
        "MATCH (c:Cluster)--(g:Gard) "
        "WHERE c.Cluster_ID = $cluster_id AND g.GARDname = $disease "
        "CREATE (c)-[:HAS_EXPERTISE ]->(e)"
    )
    session.run(query, sentence=sentence, explan=explan,cluster_id=cluster_id, disease=disease)


# Iterate over data to create Cluster and Expertise nodes
for i in data.index:
    event_properties = {
        'Cluster_ID': str(data['Community'][i]),
        'Cluster_Size': str(data['Size'][i]),
        'Evidence': str(data['Abstract'][i]),
        'Key_trems': str(data['frequent_words'][i]),
        'Llama_3_n': '',#data['Llama_3_n'][i],
    }
    create_Cluster_node(event_properties, disease)


In [ ]:
def create_relationship_cluster_researcher(community, name, affiliation, disease):
    """
    Creates a relationship between Cluster and Researcher nodes in Neo4j only if
    the Cluster node is already connected to the Gard node.
    """
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "MATCH (n1:Cluster)--(i:Gard), (n2:Researcher) "
                "WHERE i.GARDname = $disease AND n1.Cluster_ID = $community "
                "AND n2.Name = $name AND n2.Aff_name = $affiliation "
                "CREATE (n1)-[:Research_on]->(n2)"
            )
            session.run(query, community=community, name=name, affiliation=affiliation, disease=disease)

for i in clustering_.index:
    # Check if node_value is a tuple/list, otherwise skip or handle it differently
        create_relationship_cluster_researcher(
            str(clustering_['Community'][i]),
            clustering_['PI'][i],  # Researcher name
            clustering_['affiliation'][i],  # Affiliation
            disease  # Disease value (make sure this exists in your DataFrame)
        )


# The rest

In [ ]:
'''
def create_relationship_between_Researchers(n1_Name, n1_Aff_name, n2_Name, n2_Aff_name, num_colab):
   with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        # Check if the relationship already exists in either direction
        check_query = (
            "MATCH (n1:Researcher)-[r:Collaborated_With]->(n2:Researcher) "
            "WHERE (n1.Name = $n1_Name AND n1.Aff_name = $n1_Aff_name AND n2.Name = $n2_Name AND n2.Aff_name = $n2_Aff_name) "
            "OR (n2.Name = $n1_Name AND n2.Aff_name = $n1_Aff_name AND n1.Name = $n2_Name AND n1.Aff_name = $n2_Aff_name) "
            "RETURN COUNT(r) AS relationship_count"
        )

        result = session.run(check_query, n1_Name=n1_Name, n1_Aff_name=n1_Aff_name, n2_Name=n2_Name, n2_Aff_name=n2_Aff_name)
        relationship_count = result.single()["relationship_count"]

        # If the relationship doesn't exist, create it
        if relationship_count == 0:
            create_query = (
                "MATCH (n1:Researcher), (n2:Researcher) "
                "WHERE n1.Name = $n1_Name AND n1.Aff_name = $n1_Aff_name "
                "AND n2.Name = $n2_Name AND n2.Aff_name = $n2_Aff_name "
                "CREATE (n1)-[:Collaborated_With {num_colab: $num_colab}]->(n2)"
            )
            session.run(create_query, n1_Name=n1_Name, n1_Aff_name=n1_Aff_name, n2_Name=n2_Name, n2_Aff_name=n2_Aff_name, num_colab=num_colab)



for key in result_dict1:
    create_relationship_between_Researchers(key[0][0], key[0][1], key[1][0], key[1][1], result_dict1[key])
'''

In [ ]:
!pip install python-louvain
import networkx as nx
layout_algorithms = [
    nx.circular_layout,
    nx.random_layout,
    nx.spring_layout,
    nx.spectral_layout,
    nx.shell_layout,
    nx.bipartite_layout,
    nx.planar_layout,
    nx.fruchterman_reingold_layout,
    nx.kamada_kawai_layout
]

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import networkx as nx
import community
from community import community_louvain
import matplotlib.pyplot as plt
# Create a directed graph
G = nx.Graph()
# Add nodes and edges from the dictionary
for nodes, edges in result_dict.items():
    G.add_node(nodes)
    for edge in edges:
        G.add_node(edge)
        G.add_edge(nodes, edge)

# Detect communities using Louvain method
partition = community_louvain.best_partition(G)
data_table = pd.DataFrame(partition.items(), columns=['Node', 'Community'])
data_table['network']=data_table.apply(lambda x: result_dict[x['Node']] ,axis=1)
index_community_detection= list(set(data_table[data_table['network']!=set()]['Community'].values))

In [ ]:
data_table[data_table['network']!=set()]

,Node,Community,network
14,"(nan, nan)",14,"{(nan, nan)}"
15,"(Dana Hardin, nan)",15,"{(Dana Hardin, nan)}"
19,"(Jeffrey Wagener, The Childrens Hospital)",19,"{(Michael Konstan, Rainbow Babies and Children..."
20,"(Michael Konstan, Rainbow Babies and Childrens...",19,"{(Peter Hiatt, Baylor College of Medicine), (R..."
21,"(Peter Hiatt, Baylor College of Medicine)",19,"{(Michael Konstan, Rainbow Babies and Children..."
...,...,...,...
103607,"(Soergel P, NaN)",15137,"{(Dauber J, NaN), (Peterson K, NaN), (Zeevi A,..."
103608,"(Meyer K, NaN)",15137,"{(Dauber J, NaN), (Peterson K, NaN), (Zeevi A,..."
103609,"(Love R, NaN)",15137,"{(Dauber J, NaN), (Peterson K, NaN), (Zeevi A,..."
103610,"(Cornwell R, NaN)",15137,"{(Dauber J, NaN), (Peterson K, NaN), (Zeevi A,..."


In [ ]:
def merge_community_detection_and_researchers_info(data_table):
 with driver.session() as session:
  query = "MATCH (i:ClinicalTrial) RETURN i.OfficialName as OfficialName , i.OfficialAffiliation as OfficialAffiliation, i.OfficialTitle as OfficialTitle  ,i.concatenatedInterventions as concatenatedInterventions, i.BriefSummary as BriefSummary"
  results = session.run(query)
  data = [record.data() for record in results]  # Convert result to a list of dictionaries
  ClinicalTrial = pd.DataFrame(data)
  ######   Grant
  query = "MATCH (i:Grant) RETURN i.pi_name as pi_name,i.org_name as org_name,i.title as title ,i.terms as terms,i.abstract as abstract"
  results = session.run(query)
  data = [record.data() for record in results]  # Convert result to a list of dictionaries
  Grant = pd.DataFrame(data)
  ##################### pubmed
  query = "MATCH (i:Pubmed) RETURN i.fullName as fullName, i.affiliation as affiliation, i.title as title, i.concatenatedKeywords as concatenatedKeywords,i.concatenatedMeshterms as concatenatedMeshterms,i.abstractText as abstractText "
  results = session.run(query)
  data = [record.data() for record in results]  # Convert result to a list of dictionaries
  Pubmed = pd.DataFrame(data)
  ################
  ClinicalTrial['name'] = ClinicalTrial.apply(lambda row: (row['OfficialName'], row['OfficialAffiliation']), axis=1)
  Grant['name'] = Grant.apply(lambda row: (row['pi_name'], row['org_name']), axis=1)
  Pubmed['name'] = Pubmed.apply(lambda row: (row['fullName'], row['affiliation']), axis=1)
  ClinicalTrial['BriefSummary']=ClinicalTrial['OfficialTitle']+ClinicalTrial['concatenatedInterventions']+ClinicalTrial['BriefSummary']
  Grant['BriefSummary']=Grant['title']+Grant['terms']+Grant['abstract']
  Pubmed['abstractText']=Pubmed['title']+Pubmed['concatenatedKeywords']+Pubmed['concatenatedMeshterms']+Pubmed['abstractText']
  ClinicalTrial['Terms']=ClinicalTrial['concatenatedInterventions']
  Grant['Terms']=Grant['terms']
  Pubmed['Terms']=Pubmed['concatenatedKeywords']+Pubmed['concatenatedMeshterms']
  #df1=ClinicalTrial[['i.OfficialName',  'c.BriefSummary', 'i.OfficialAffiliation','name']]
  df1=ClinicalTrial[['OfficialName',  'BriefSummary','OfficialAffiliation', 'name','Terms']]
  df2=Grant[['pi_name', 'abstract', 'org_name','name','Terms']]
  df3=Pubmed[['fullName',  'abstractText', 'affiliation','name','Terms']]
  # Rename columns to avoid conflicts
  df1.columns = ['PI','Abstract', 'affiliation','name','Terms']
  df2.columns = ['PI','Abstract', 'affiliation','name','Terms']
  df3.columns = ['PI','Abstract', 'affiliation','name','Terms']
  #df2['Location']=  df2['locationCity'] +' , '+df2['Location']
  result_df = pd.concat([df1  , df2, df3])
  #result_df = pd.concat([df1,  df3])
  result_df=result_df.drop_duplicates()
  #################################################### merging with c
  merged_data1 = pd.merge(result_df,data_table , left_on='name', right_on='Node', how='outer')
  concatenated_data = merged_data1.drop_duplicates(subset=['name'])
  concatenated_data.reset_index(drop=True, inplace=True)
  concatenated_data=concatenated_data.drop(['name'],axis=1)
  return concatenated_data
concatenated_data= merge_community_detection_and_researchers_info(data_table)

In [ ]:
concatenated_data

,PI,Abstract,affiliation,Terms,Node,Community,network
0,NaN,NaN,NaN,None,"(nan, nan)",14.0,"{(nan, nan)}"
1,Dearbhla Hull,NaN,Zambon SpA,None,"(Dearbhla Hull, Zambon SpA)",749.0,{}
2,Anthony Bier,NaN,Nutrasource Pharmaceutical and Nutraceutical S...,None,"(Anthony Bier, Nutrasource Pharmaceutical and ...",1073.0,{}
3,Mina Pastagia,NaN,"Armata Pharmaceuticals, Inc.",None,"(Mina Pastagia, Armata Pharmaceuticals, Inc.)",958.0,{}
4,Mary C. Drinane,NaN,Dartmouth-Hitchcock Medical Center,None,"(Mary C. Drinane, Dartmouth-Hitchcock Medical ...",1043.0,{}
...,...,...,...,...,...,...,...
96592,"Eiberg, H., Schmiegelow, K., Tsui, L.-C., Buch...",NaN,None,NaN,NaN,NaN,NaN
96593,"Eiberg, H., Mohr, J., Nielsen, L. S.",NaN,None,NaN,NaN,NaN,NaN
96594,"Danks, D. M., Allan, J., Anderson, C. M.",NaN,None,NaN,NaN,NaN,NaN
96595,"Cutting, G. R.",NaN,None,NaN,NaN,NaN,NaN


In [ ]:
clusetring_1=concatenated_data[concatenated_data['Community'].isin(index_community_detection)]
clustering_2=concatenated_data[~concatenated_data['Community'].isin(index_community_detection)]

In [ ]:
clustering_2

,PI,Abstract,affiliation,Terms,Node,Community,network
1,Dearbhla Hull,NaN,Zambon SpA,None,"(Dearbhla Hull, Zambon SpA)",749.0,{}
2,Anthony Bier,NaN,Nutrasource Pharmaceutical and Nutraceutical S...,None,"(Anthony Bier, Nutrasource Pharmaceutical and ...",1073.0,{}
3,Mina Pastagia,NaN,"Armata Pharmaceuticals, Inc.",None,"(Mina Pastagia, Armata Pharmaceuticals, Inc.)",958.0,{}
4,Mary C. Drinane,NaN,Dartmouth-Hitchcock Medical Center,None,"(Mary C. Drinane, Dartmouth-Hitchcock Medical ...",1043.0,{}
7,H. Frank Farmer,NaN,"Covance CRU, Inc.",None,"(H. Frank Farmer, Covance CRU, Inc.)",254.0,{}
...,...,...,...,...,...,...,...
96592,"Eiberg, H., Schmiegelow, K., Tsui, L.-C., Buch...",NaN,None,NaN,NaN,NaN,NaN
96593,"Eiberg, H., Mohr, J., Nielsen, L. S.",NaN,None,NaN,NaN,NaN,NaN
96594,"Danks, D. M., Allan, J., Anderson, C. M.",NaN,None,NaN,NaN,NaN,NaN
96595,"Cutting, G. R.",NaN,None,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

def name_clusters(df, text_column, cluster_column, num_clusters=5):
    # Extract abstracts from the specified column and handle NaN values
    abstracts = df[text_column].fillna("").astype(str).tolist()

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(abstracts)

    # KMeans clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    clusters = kmeans.fit_predict(tfidf_matrix)

    # Assign cluster labels to the DataFrame
    df[cluster_column] = clusters

    return df

# Assuming you have a DataFrame `clustering_2`
clustering_2_ = name_clusters(clustering_2, 'Abstract', 'Community', num_clusters=300)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans  # Use MiniBatchKMeans for faster clustering
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD  # Dimensionality reduction to speed up vectorization

def name_clusters(df, text_column, cluster_column, num_clusters=5, n_components=100):
    # Extract abstracts from the specified column and handle NaN values
    abstracts = df[text_column].fillna("").astype(str).tolist()

    # TF-IDF vectorization with ngram_range (optimize based on your dataset)
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))  # Bi-grams may help for better context
    tfidf_matrix = vectorizer.fit_transform(abstracts)

    # Optionally reduce dimensionality to speed up clustering
    if n_components:
        svd = TruncatedSVD(n_components=n_components, random_state=42)
        tfidf_matrix = svd.fit_transform(tfidf_matrix)

    # MiniBatchKMeans clustering (faster than KMeans)
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=42, n_init=10, batch_size=1000)
    clusters = kmeans.fit_predict(tfidf_matrix)

    # Assign cluster labels to the DataFrame
    df[cluster_column] = clusters

    return df

# Assuming you have a DataFrame `clustering_2`
clustering_2_ = name_clusters(clustering_2, 'Abstract', 'Community', num_clusters=300, n_components=100)
#clustering_2_.loc[:, 'Community'] = clustering_2_['Community'] + 100000000000
clustering_2_['Community']=clustering_2_.apply(lambda x: x['Community']+100000000000, axis=1)
clustering_=pd.concat([clusetring_1,clustering_2_])
community_mapping = {val: idx + 1 for idx, val in enumerate(clustering_['Community'].unique())}
# Map the original 'Community' values to the new values
clustering_['Community'] = clustering_['Community'].map(community_mapping)


C:\Users\valinejadj2\AppData\Local\Temp\ipykernel_25404\1852488632.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cluster_column] = clusters
C:\Users\valinejadj2\AppData\Local\Temp\ipykernel_25404\1852488632.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clustering_2_['Community']=clustering_2_.apply(lambda x: x['Community']+100000000000, axis=1)


In [ ]:
clustering_

,PI,Abstract,affiliation,Terms,Node,Community,network
0,NaN,NaN,NaN,None,"(nan, nan)",1,"{(nan, nan)}"
5,Nicolas Roche,NaN,APHP- Hopital Cochin,None,"(Nicolas Roche, APHP- Hopital Cochin)",2,"{(Thierry Perez, CHRU de Lille / Hôpital Calme..."
6,Thierry Perez,NaN,CHRU de Lille / Hôpital Calmette - France,None,"(Thierry Perez, CHRU de Lille / Hôpital Calmet...",2,"{(Nicolas Roche, APHP- Hopital Cochin)}"
19,Remi Rabasa-Lhoret,NaN,Institut de Recherches Cliniques de Montreal,None,"(Remi Rabasa-Lhoret, Institut de Recherches Cl...",3,"{(Yves Berthiaume, Institut de Recherches Clin..."
20,Yves Berthiaume,NaN,Institut de Recherches Cliniques de Montreal,None,"(Yves Berthiaume, Institut de Recherches Clini...",3,"{(Remi Rabasa-Lhoret, Institut de Recherches C..."
...,...,...,...,...,...,...,...
96592,"Eiberg, H., Schmiegelow, K., Tsui, L.-C., Buch...",NaN,None,NaN,NaN,276,NaN
96593,"Eiberg, H., Mohr, J., Nielsen, L. S.",NaN,None,NaN,NaN,276,NaN
96594,"Danks, D. M., Allan, J., Anderson, C. M.",NaN,None,NaN,NaN,276,NaN
96595,"Cutting, G. R.",NaN,None,NaN,NaN,276,NaN


In [ ]:
import pandas as pd

def sum_strings(series):
    # Convert all items to string, handling NaNs
    return ', '.join(str(item) for item in series if pd.notna(item))

# Function to clean and flatten terms
def sum_terms(series):
    result = []
    for item in series:
        if isinstance(item, list):  # Check if item is a list
            for subitem in item:
                if isinstance(subitem, str):  # Ensure it's a string
                    subitem_cleaned = subitem.strip('[]')  # Remove surrounding brackets
                    result.extend(subitem_cleaned.split(','))  # Split and add terms
        elif pd.notna(item):  # Handle non-list, non-NaN values
            item_cleaned = str(item).strip('[]')  # Convert to string and remove brackets
            result.extend(item_cleaned.split(','))  # Split and add terms
    # Remove extra whitespace from terms
    return [term.strip() for term in result if term.strip()]

# Group by 'Community' and aggregate columns using sum_strings and sum_terms functions
result = clustering_.groupby('Community').agg(
    Abstract=('Abstract', sum_strings),
    Terms=('Terms', sum_terms),
    Size=('Community', 'size')  # Count of occurrences
).reset_index()

result

,Community,Abstract,Terms,Size
0,1,,[],1
1,2,,[],2
2,3,,[],2
3,4,,[],2
4,5,,[],2
...,...,...,...,...
278,279,DESCRIPTION (provided by applicant): This m...,"[Lung diseases, Research Training, Lung diseas...",27
279,280,DESCRIPTION (provided by applicant): Molecular...,"[Address, Affinity, Alzheimer's Disease, Apica...",33
280,281,[unreadable] DESCRIPTION (provided by applic...,"[immune tolerance /unresponsiveness, model, re...",54
281,282,DESCRIPTION (provided by applicant): This is a...,"[Childhood, Research Training, Translational R...",6


In [ ]:
num_nulls = result['Abstract'].isnull().sum()
print(f"Number of nulls in 'c.BriefSummary': {num_nulls}")
num_empty_strings = (result['Abstract'] == '').sum()
print(f"Number of empty strings in 'c.BriefSummary': {num_empty_strings}")

Number of nulls in 'c.BriefSummary': 0
Number of empty strings in 'c.BriefSummary': 130


In [ ]:
result['Size'].max()

93105

In [ ]:
result.to_csv('LLM_input.csv')

Run LLM

In [ ]:
import pandas as pd
result = pd.read_csv('C:/Users/Desktop/data.csv')
result

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Community,Abstract,Terms,Size,frequent_words,Mistral,Llama_3_n,term_1n,term_2n,term_3n
0,0,0,0,0,1,Development of a New Tool for Dyspnea Measurem...,['Cross sectional psychometric evaluation of a...,2,Cross sectional psychometric evaluation of a s...,1. Psychometrics: The study focused on the psy...,1. Psychometrics\n2. Questionnaire Design\n3. ...,Psychometrics,Questionnaire Design,Dyspnea Measurement
1,1,1,1,1,2,The Effects of a Low Glycemic Load Diet on Dys...,"['Low Glycemic Load Diet', 'Low Glycemic Load ...",2,Low Glycemic Load Diet,1. Glycemic Control\n2. Nutrition Management\n...,1. Glycemic Index\n2. Glycemic Load\n3. Nutrit...,Glycemic Index,Glycemic Load,Nutritional Assessment
2,2,2,2,2,3,"A Phase 1/2 Study to Evaluate the Safety, Tole...","['PTI-808, Placebo, PTI-428, PTI-801', 'PTI-80...",2,"PTI-808, Placebo, PTI-428, PTI-801",1. Pharmacokinetics: The study will evaluate t...,1. Pharmacokinetics\n2. Pharmacodynamics\n3. D...,Pharmacokinetics,Pharmacodynamics,Drug-drug interactions
3,3,3,3,3,4,In Vitro Comparison of Continuous and Breath-s...,['Nebulization of Amikacin during NIV (RR: 25 ...,7,Nebulization of Amikacin during NIV (RR: 25 cy...,1. Nebulization\n2. Inhaled and Lost Doses\n3....,"Keywords: Nebulization, Amikacin, Non-invasive...",Nebulization,Amikacin,invasive ventilation
4,4,4,4,4,5,Intraduodenal Aspiration Study to Assess the B...,"['Pancrelipase', 'Pancrelipase']",2,Pancrelipase,1. Pancreatic Enzyme Replacement Therapy (PERT...,1. Pancrelipase\n2. Pancreatic Insufficiency\n...,Pancrelipase,Pancreatic Insufficiency,Bioavailability
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,108,108,108,108,109,Project Summary/Abstract Cilia dysfunction lea...,"['[Actins, Adaptor Signaling Protein, Addres...",4,"[Actins, Adaptor Signaling Protein, Address,...",1. Microtubule dynamics\n2. Protein traffickin...,NaN,NaN,NaN,NaN
109,109,109,109,109,110,...,"['[bacteria infection mechanism, respiratory i...",7,"[16S ribosomal RNA sequencing, Adopted, Adop...",1. Bioinformatics - The use of computational t...,NaN,NaN,NaN,NaN
110,110,110,110,110,111,DESCRIPTION (provided by applicant): Cystic Fi...,"['[absorption, Affect, Animal Model, Area, Bac...",11,"[absorption, Affect, Animal Model, Area, Bacte...",1. Comparative Pathology: The study of the dif...,NaN,NaN,NaN,NaN
111,111,111,111,111,112,This project will test the hypothesis that som...,"['[lung disorder, chloride channels, protein d...",3,"[lung disorder, chloride channels, protein deg...","1. Molecular biology: Genetic engineering, gen...",NaN,NaN,NaN,NaN


In [ ]:
def create_Cluster_node(properties):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            query = (
                "CREATE (Cl:Cluster {"
                'Cluster_ID: $Cluster_ID,'
                'Cluster_Size: $Cluster_Size,'
                'Evidence: $Evidence,'
                'Key_trems: $Key_trems'
                "})"
            )
            session.run(query, **properties)

for i in result.index:
    event_properties = {
        'Cluster_ID': str(result['Community'][i]),
        'Cluster_Size': str(result['Size'][i]),
        'Evidence': str(result['Abstract'][i]),
        'Key_trems': str(result['frequent_words'][i]),
    }
    create_Cluster_node(event_properties)


In [ ]:
'''
event_properties = {'GARDID': 'GARD0006233' ,'GARDname': 'Cystic fibrosis'}
create_Gard_node(event_properties)
'''

def create_relationship_between_nodes(node1_label, node2_label, relationship_type, GARDname_):
   with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        query = (
            f"MATCH (n1:{node1_label}), (n2:{node2_label}) "
            f"WHERE n2.GARDname = '{GARDname_}' "  # Enclose GARDname_ in single quotes
            f"CREATE (n1)-[:{relationship_type}]->(n2)"
        )
        session.run(query)

# Example usage:
create_relationship_between_nodes('Cluster', 'Gard', "Cluster_to_GARD", 'Cystic fibrosis')

In [ ]:
def create_relationship_cluster_researcher(community, name, affiliation):
  with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        query = (
            "MATCH (n1:Cluster), (n2:Researcher) "
            "WHERE n1.Cluster_ID = $community AND n2.Name = $name AND n2.Aff_name = $affiliation "
            "CREATE (n1)-[:Related_cluster]->(n2)"
        )
        session.run(query, community=community, name=name, affiliation=affiliation)

for i in clustering_.index:
    node_value = clustering_['Node'][i]
    # Check if node_value is a tuple/list, otherwise skip or handle it differently
    if isinstance(node_value, (tuple, list)) and len(node_value) >= 2:
        create_relationship_cluster_researcher(
            str(clustering_['Community'][i]),
            node_value[0],  # Researcher name
            node_value[1]   # Affiliation
        )

In [ ]:
type(result['term_3n'][60]) != str

True

In [ ]:
import nltk
import re

def extract_terms(sentence):
    # Check if the sentence is a valid string
    if not isinstance(sentence, str):return []  # Return empty list if not a string
    # Regular expression to match the terms
    pattern = r'\d+\.\s*(.*?)(?:\n|<\|end_of_text\|>|\s*$)'
    # Find all matches in the sentence
    terms = re.findall(pattern, sentence)
    # Return only the first four terms, or all if less than four
    return terms[:4]

def extract_and_create_expertise_nodes():
  with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        # Query to retrieve all Cluster nodes with Evidence
        query = "MATCH (c:Cluster) RETURN c.Cluster_ID AS cluster_id, c.Evidence AS evidence"
        results = session.run(query)

        for record in results:
          cluster_id = record['cluster_id']
          if  cluster_id:
            try:
              evidence = result['Llama_3_n'][result['Community'].astype(str) == str(cluster_id)].iloc[0] #record['evidence']
            except: evidence=None
              # Check if evidence is not None
            if evidence:
                # Split evidence into sentences
                sentences =  [result['term_1n'],result['term_2n'],result['term_3n']]  #extract_terms(evidence)

                for sentence in sentences:
                   if type(sentence) == str and sentence != 'None':
                    # Create a new Expertise node for each sentence
                    create_expertise_node(session, sentence, cluster_id)

def create_expertise_node(session, sentence, cluster_id):
    # Create a new Expertise node with a relationship to the Cluster
    query = (
        "CREATE (e:Expertise {Summarized_expertise: $sentence}) "
        "WITH e "
        "MATCH (c:Cluster {Cluster_ID: $cluster_id}) "
        "CREATE (c)-[:HAS_EXPERTISE]->(e)"
    )
    session.run(query, sentence=sentence, cluster_id=cluster_id)

# Call the function
extract_and_create_expertise_nodes()

In [ ]:
import nltk
import re

def extract_terms(sentence):
    # Check if the sentence is a valid string
    if not isinstance(sentence, str):return []  # Return empty list if not a string
    # Regular expression to match the terms
    pattern = r'\d+\.\s*(.*?)(?:\n|<\|end_of_text\|>|\s*$)'
    # Find all matches in the sentence
    terms = re.findall(pattern, sentence)
    # Return only the first four terms, or all if less than four
    return terms[:4]

def extract_and_create_expertise_nodes():
  with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        # Query to retrieve all Cluster nodes with Evidence
        query = "MATCH (c:Cluster) RETURN c.Cluster_ID AS cluster_id, c.Evidence AS evidence"
        results = session.run(query)

        for i in result.index:
                cluster_id = str(result['Community'][i])
                sentences =  [result['term_1n'][i],result['term_2n'][i],result['term_3n'][i]]  #extract_terms(evidence)
                for sentence in sentences:
                   if type(sentence) == str and sentence != 'None':
                    # Create a new Expertise node for each sentence
                    create_expertise_node(session, sentence, cluster_id)

def create_expertise_node(session, sentence, cluster_id):
    # Create a new Expertise node with a relationship to the Cluster
    query = (
        "CREATE (e:Expertise {Summarized_expertise: $sentence}) "

    )
    session.run(query, sentence=sentence, cluster_id=cluster_id)

# Call the function
extract_and_create_expertise_nodes()

In [ ]:
def export_data():
    with driver.session() as session:
        # Adjust the path and format as needed (CSV, JSON, etc.)
        export_query = """
        CALL apoc.export.csv.all("file:///exported_data.csv", {})
        """
        session.run(export_query)
        print("Data export completed.")

# Run the export
export_data()